<a href="https://colab.research.google.com/github/Sumit-Dwivedi/SHM-ML-model-for-pipelines/blob/main/SHM_For_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # Added for visualization
import os
from sklearn.preprocessing import StandardScaler # Keep StandardScaler
from sklearn.model_selection import train_test_split

# Set random seed for reproducibility
np.random.seed(42)

# Define category mapping for clarity and visualization
DAMAGE_CATEGORIES = {
    0: 'Normal',
    1: 'Micro',
    2: 'Minor',
    3: 'Major'
}

def load_frequency_data(file_path, selected_sheets):
    """
    Load data from multiple sheets in a single Excel file.
    """
    frequency_data = {}
    try:
        xls = pd.ExcelFile(file_path)
        # Assume sheet names directly map or adjust mapping if needed
        frequency_mapping = {name: name for name in xls.sheet_names if name in selected_sheets}

        for sheet_name in selected_sheets:
            if sheet_name in frequency_mapping:
                df = pd.read_excel(file_path, sheet_name=sheet_name)
                freq = frequency_mapping[sheet_name]
                print(f"Loaded {freq} data: {df.shape[0]} rows, {df.shape[1]} columns")
                frequency_data[freq] = df
            else:
                 print(f"Warning: Selected sheet '{sheet_name}' not found in the Excel file.")

    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"Error loading data: {e}")
        return None
    return frequency_data

def add_acquisition_info(dataframes, acquisition_ranges):
    """
    Add acquisition number and damage category label (0-3) to dataframes.

    Categorization Logic (based on acquisition number 'i'):
    - Normal (0): i < damage_start
    - Micro (1): damage_start <= i <= micro_boundary
    - Minor (2): micro_boundary < i <= minor_boundary
    - Major (3): i > minor_boundary
    """
    updated_dfs = {}

    # --- Define category boundaries based on acquisition_ranges ---
    damage_start = acquisition_ranges['damage_start']
    acq_end = acquisition_ranges['end']
    total_damaged_acqs = acq_end - damage_start + 1

    if total_damaged_acqs <= 0:
        print("Warning: No damaged acquisitions based on provided range.")
        micro_boundary = damage_start - 1 # No micro/minor/major range
        minor_boundary = damage_start - 1
    elif total_damaged_acqs < 3:
         print("Warning: Very few damaged acquisitions (<3), assigning all to 'Micro'.")
         micro_boundary = acq_end # All damaged fall into micro category (up to end)
         minor_boundary = acq_end # No distinct minor/major range
    else:
        # Divide damaged range into three roughly equal parts using floor division
        # Boundary is the *last* acquisition number in the category
        acqs_per_cat = total_damaged_acqs // 3
        micro_boundary = damage_start + acqs_per_cat - 1
        minor_boundary = micro_boundary + acqs_per_cat
        # Major starts after minor_boundary

    print(f"Categorization Ranges: Normal(<{damage_start}), Micro({damage_start}-{micro_boundary}), Minor({micro_boundary+1}-{minor_boundary}), Major(>{minor_boundary})")
    # --- End Category Definition ---


    for freq, df in dataframes.items():
        # Assuming each acquisition might have slightly different numbers of measurements
        # Estimate based on total rows and total acquisitions for array pre-allocation
        total_rows_in_df = df.shape[0]
        total_acquisitions_in_range = acquisition_ranges['end'] - acquisition_ranges['start'] + 1
        if total_acquisitions_in_range <= 0:
            print(f"Error for {freq}: Invalid acquisition range.")
            continue
        # Use floating point division for average, then ceiling for initial estimate
        avg_measurements = total_rows_in_df / total_acquisitions_in_range
        est_measurements_per_acq = int(np.ceil(avg_measurements)) if avg_measurements > 0 else 1
        print(f"Estimating ~{est_measurements_per_acq} measurements per acquisition for {freq}.")


        all_acquisitions = []
        # *** MODIFICATION START: Use all_damage_categories ***
        all_damage_categories = []
        # *** MODIFICATION END ***

        acq_row_counts = {} # Track actual rows per acq if needed later

        current_row_index = 0
        for i in range(acquisition_ranges['start'], acquisition_ranges['end'] + 1):

            # --- Determine Damage Category for acquisition 'i' ---
            if i < damage_start:
                category = 0  # Normal
            elif i <= micro_boundary:
                category = 1  # Micro
            elif i <= minor_boundary:
                category = 2  # Minor
            else:
                category = 3  # Major
            # --- End Damage Category Determination ---

            # Find rows corresponding to the current acquisition number 'i'
            # This assumes the original Excel data might be ordered or requires search
            # For simplicity based on previous runs, assume contiguous blocks based on estimated size.
            # THIS IS A POTENTIAL SOURCE OF ERROR if measurements_per_acquisition varies significantly.
            # A more robust way would be to pre-process the Excel to *have* an acquisition column
            # or use the grouping logic from extract_features here.
            # Using estimated block size for now:
            start_index = current_row_index
            # Make sure end_index doesn't exceed total rows
            end_index = min(start_index + est_measurements_per_acq, total_rows_in_df)
            num_measurements_this_acq = end_index - start_index

            if num_measurements_this_acq <= 0 and current_row_index < total_rows_in_df:
                 # If estimation leads to zero measurements, take at least one if rows remain
                 num_measurements_this_acq = 1
                 end_index = start_index + 1

            # Repeat the acquisition number and category for the determined number of measurements
            all_acquisitions.extend([i] * num_measurements_this_acq)
            # *** MODIFICATION START: Extend the correct list ***
            all_damage_categories.extend([category] * num_measurements_this_acq)
            # *** MODIFICATION END ***

            current_row_index = end_index # Move to the next block start
            acq_row_counts[i] = num_measurements_this_acq # Store count

            if current_row_index >= total_rows_in_df:
                 if i < acquisition_ranges['end']:
                      print(f"Warning for {freq}: Reached end of DataFrame rows ({total_rows_in_df}) before processing all expected acquisitions (up to {acquisition_ranges['end']}). Stopping assignment at acq {i}.")
                 break # Stop if we've assigned labels for all rows

        # --- Adjust lists if total length doesn't match DataFrame ---
        # This part might be less necessary if the block estimation above is accurate enough,
        # but keep it as a safety net.
        current_len = len(all_acquisitions)
        if current_len != total_rows_in_df:
            print(f"Warning for {freq}: Final assigned rows ({current_len}) don't match DataFrame rows ({total_rows_in_df}). Adjusting lists.")
            if current_len > total_rows_in_df:
                all_acquisitions = all_acquisitions[:total_rows_in_df]
                # *** MODIFICATION START: Adjust correct list ***
                all_damage_categories = all_damage_categories[:total_rows_in_df]
                # *** MODIFICATION END ***
            else: # current_len < total_rows_in_df
                remaining = total_rows_in_df - current_len
                last_acq = all_acquisitions[-1] if all_acquisitions else acquisition_ranges['end'] # Use last assigned or end
                # Determine category for the last acquisition
                last_cat = 0
                if last_acq > minor_boundary: last_cat = 3
                elif last_acq > micro_boundary: last_cat = 2
                elif last_acq >= damage_start: last_cat = 1

                all_acquisitions.extend([last_acq] * remaining)
                # *** MODIFICATION START: Extend correct list ***
                all_damage_categories.extend([last_cat] * remaining)
                # *** MODIFICATION END ***
        # --- End Adjustment ---


        # --- Add new columns ---
        df_copy = df.copy()
        df_copy['acquisition_number'] = all_acquisitions
        # *** MODIFICATION START: Use new column name ***
        df_copy['damage_category'] = all_damage_categories
        # *** MODIFICATION END ***

        updated_dfs[freq] = df_copy
        # *** MODIFICATION START: Print new category distribution ***
        print(f" -> Added info for {freq}. Category distribution: {df_copy['damage_category'].value_counts().sort_index()}")
        # *** MODIFICATION END ***

    return updated_dfs

def extract_features(dataframes):
    """
    Extract relevant features from the ultrasonic guided wave data.
    """
    feature_dfs = {}

    for freq, df in dataframes.items():
        # *** MODIFICATION START: Check for new column name ***
        if not {'acquisition_number', 'damage_category'}.issubset(df.columns):
             print(f"Error: Missing 'acquisition_number' or 'damage_category' in DataFrame for {freq}. Skipping.")
             continue
        # *** MODIFICATION END ***
        if df.shape[1] < 5: # Dist, Tors, Flex, Ind?, Acq, Cat
             print(f"Warning: DataFrame for {freq} has fewer columns than expected. Check column indexing.")

        grouped = df.groupby('acquisition_number')
        features = []

        for acq_num, group in grouped:
            if group.empty: continue
            # Assuming columns are [distance?, torsional, flexural, indicator?, acq, cat]
            torsional_data = group.iloc[:, 1].values  # Torsional column
            flexural_data = group.iloc[:, 2].values   # Flexural column

            feature_dict = {
                'acquisition_number': acq_num,
                # *** MODIFICATION START: Use new column name ***
                'damage_category': group['damage_category'].iloc[0],
                # *** MODIFICATION END ***

                # Torsional wave features
                'torsional_mean': np.mean(torsional_data),
                'torsional_std': np.std(torsional_data),
                'torsional_max': np.max(torsional_data),
                'torsional_min': np.min(torsional_data),
                'torsional_peak_to_peak': np.ptp(torsional_data),
                'torsional_rms': np.sqrt(np.mean(np.square(torsional_data))),
                'torsional_kurtosis': pd.Series(torsional_data).kurtosis(), # Use pandas

                # Flexural wave features
                'flexural_mean': np.mean(flexural_data),
                'flexural_std': np.std(flexural_data),
                'flexural_max': np.max(flexural_data),
                'flexural_min': np.min(flexural_data),
                'flexural_peak_to_peak': np.ptp(flexural_data),
                'flexural_rms': np.sqrt(np.mean(np.square(flexural_data))),
                'flexural_kurtosis': pd.Series(flexural_data).kurtosis(), # Use pandas

                # Signal energy features
                'torsional_energy': np.sum(np.square(torsional_data)),
                'flexural_energy': np.sum(np.square(flexural_data)),
                'energy_ratio': (np.sum(np.square(torsional_data)) /
                                (np.sum(np.square(flexural_data)) + 1e-9)) # Add epsilon
            }
            features.append(feature_dict)

        feature_df = pd.DataFrame(features) if features else pd.DataFrame()
        feature_dfs[freq] = feature_df
        if not feature_df.empty:
            print(f" -> Extracted features for {freq}: {feature_df.shape[0]} acquisitions.")
        else:
            print(f" -> No features extracted for {freq}.")


    return feature_dfs

def normalize_features(feature_dfs):
    """
    Normalize features using StandardScaler.
    """
    normalized_dfs = {}
    scalers = {}

    for freq, df in feature_dfs.items():
        if df.empty:
             print(f"Skipping normalization for empty DataFrame {freq}")
             normalized_dfs[freq] = df
             scalers[freq] = None
             continue
        # *** MODIFICATION START: Use new column name ***
        labels = df[['acquisition_number', 'damage_category']]
        features = df.drop(['acquisition_number', 'damage_category'], axis=1)
        # *** MODIFICATION END ***

        if features.empty:
             print(f"No features to normalize for {freq}. Returning only labels.")
             normalized_dfs[freq] = labels.reset_index(drop=True)
             scalers[freq] = None
             continue

        scaler = StandardScaler()
        try:
            normalized_features = scaler.fit_transform(features)
        except ValueError as e:
             print(f"Error scaling features for {freq}: {e}. Check data.")
             normalized_dfs[freq] = df # Return unnormalized as fallback
             scalers[freq] = None
             continue


        normalized_df = pd.DataFrame(normalized_features, columns=features.columns, index=features.index) # Keep index

        # *** MODIFICATION START: Align using index ***
        normalized_df = pd.concat([labels, normalized_df], axis=1)
        # *** MODIFICATION END ***

        normalized_dfs[freq] = normalized_df
        scalers[freq] = scaler
        print(f" -> Normalized features for {freq}.")

    return normalized_dfs, scalers

def split_data(feature_dfs, test_size=0.2, validation_size=0.15): # Renamed arg
    """
    Split data into training, validation, and test sets, stratified by damage category.
    """
    split_data_dict = {}

    for freq, df in feature_dfs.items():
        if df.empty:
             print(f"Skipping split for empty DataFrame {freq}")
             split_data_dict[freq] = {'train': pd.DataFrame(), 'val': pd.DataFrame(), 'test': pd.DataFrame()}
             continue

        # *** MODIFICATION START: Use new column name ***
        y = df['damage_category']
        # Check for sufficient samples for stratification
        min_samples = y.value_counts().min()
        n_splits_needed = 2 # For train/test and then train/val
        if min_samples < n_splits_needed:
            print(f"Warning for {freq}: Cannot stratify split (min samples={min_samples} < {n_splits_needed}). Using non-stratified split.")
            stratify_y = None
        else:
            stratify_y = y

        X = df.drop(['damage_category'], axis=1) # Keep acq number for now if needed
        # *** MODIFICATION END ***


        # First split: separate test set
        try:
            X_train_val, X_test, y_train_val, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42, stratify=stratify_y
            )
        except ValueError as e:
            print(f"Initial split error (stratify={stratify_y is not None}) for {freq}: {e}. Retrying without stratification.")
            X_train_val, X_test, y_train_val, y_test = train_test_split(
                X, y, test_size=test_size, random_state=42, stratify=None
            )

        # Second split: separate validation set
        # Check stratification for validation split
        if stratify_y is not None:
             min_samples_tv = y_train_val.value_counts().min()
             if min_samples_tv < n_splits_needed:
                  print(f"Warning for {freq}: Cannot stratify validation split (min samples={min_samples_tv} < {n_splits_needed}). Using non-stratified split.")
                  stratify_y_val = None
             else:
                  stratify_y_val = y_train_val
        else:
             stratify_y_val = None

        val_ratio = validation_size / (1.0 - test_size)  # Adjusted validation ratio
        try:
            X_train, X_val, y_train, y_val = train_test_split(
                X_train_val, y_train_val, test_size=val_ratio, random_state=42, stratify=stratify_y_val
            )
        except ValueError as e:
            print(f"Validation split error (stratify={stratify_y_val is not None}) for {freq}: {e}. Retrying without stratification.")
            X_train, X_val, y_train, y_val = train_test_split(
                X_train_val, y_train_val, test_size=val_ratio, random_state=42, stratify=None
            )


        # Restore the target variable and acquisition number
        # *** MODIFICATION START: Use new column name ***
        train_df = X_train.copy()
        train_df['damage_category'] = y_train

        val_df = X_val.copy()
        val_df['damage_category'] = y_val

        test_df = X_test.copy()
        test_df['damage_category'] = y_test
        # *** MODIFICATION END ***

        # Re-add acquisition number if it was dropped earlier (it was kept in X here)
        # If X included 'acquisition_number', it's already in train_df, val_df, test_df

        split_data_dict[freq] = {
            'train': train_df,
            'val': val_df,
            'test': test_df
        }
        # *** MODIFICATION START: Print new category distribution ***
        print(f" -> Data split for {freq}: Train({len(train_df)}), Val({len(val_df)}), Test({len(test_df)})")
        if not train_df.empty: print(f"    Train distribution: {train_df['damage_category'].value_counts().sort_index()}")
        if not val_df.empty: print(f"    Val distribution:   {val_df['damage_category'].value_counts().sort_index()}")
        if not test_df.empty: print(f"    Test distribution:  {test_df['damage_category'].value_counts().sort_index()}")
        # *** MODIFICATION END ***

    return split_data_dict

def create_time_windows(dataframes, window_size=5, step=1):
    """
    Create time windows for sequential data analysis using the split feature DataFrames.
    """
    windowed_data = {}

    for freq, df_dict in dataframes.items():
        windowed_data[freq] = {} # Initialize frequency entry
        for split_name, df in df_dict.items():
             # Ensure df is not empty and has the required columns before processing
             if df.empty or not {'acquisition_number', 'damage_category'}.issubset(df.columns):
                 print(f"Skipping window creation for {freq} - {split_name}: DataFrame empty or missing required columns.")
                 windowed_data[freq][split_name] = (np.array([]), np.array([])) # Assign empty arrays
                 continue

             windowed_result = create_windows_for_dataset(df, window_size, step)
             windowed_data[freq][split_name] = windowed_result
             # *** MODIFICATION START: Print shapes ***
             if windowed_result[0].size > 0:
                 print(f" -> Created windows for {freq} - {split_name}: X shape {windowed_result[0].shape}, y shape {windowed_result[1].shape}")
             else:
                 print(f" -> No windows created for {freq} - {split_name}.")
            # *** MODIFICATION END ***

    return windowed_data


def create_windows_for_dataset(df, window_size, step):
    """
    Helper function to create windows for a single dataset (train, val, or test).
    """
    # Ensure df is not empty and has required columns
    if df.empty or not {'acquisition_number', 'damage_category'}.issubset(df.columns):
         return np.array([]), np.array([])

    # Sort by acquisition number first is crucial for time windows
    df = df.sort_values(by='acquisition_number').reset_index(drop=True)

    # Get unique acquisition numbers in sorted order
    acquisitions = df['acquisition_number'].unique()
    acquisitions.sort()

    # Features to include in the window
    # *** MODIFICATION START: Use new column name ***
    feature_cols = df.drop(['acquisition_number', 'damage_category'], axis=1).columns
    # *** MODIFICATION END ***
    if not list(feature_cols): # Check if feature_cols is empty
        print("Warning: No feature columns found for window creation.")
        return np.array([]), np.array([])

    X_windows = []
    y_windows = []

    # Iterate through unique acquisitions to define window start points
    for i in range(0, len(acquisitions) - window_size + 1, step):
        window_acquisitions = acquisitions[i : i + window_size]

        # Select rows from the dataframe corresponding to these acquisitions
        window_data = df[df['acquisition_number'].isin(window_acquisitions)]

        # Ensure data is sorted by acquisition within the window slice
        window_data = window_data.sort_values(by='acquisition_number')

        # Check if we have exactly one row per acquisition number in the window
        if len(window_data) != window_size:
            # This indicates missing acquisitions or duplicate entries within the expected range
            # print(f"Skipping window at acq {acquisitions[i]}: Expected {window_size} rows, found {len(window_data)}.")
            continue

        # Extract features for the window
        features = window_data[feature_cols].values

        # Get the label of the last acquisition in the window
        last_acq = window_acquisitions[-1]
        try:
            # *** MODIFICATION START: Use new column name ***
            label = window_data[window_data['acquisition_number'] == last_acq]['damage_category'].iloc[0]
            # *** MODIFICATION END ***
        except IndexError:
            # Should not happen if len(window_data) == window_size, but safety check
            # print(f"Could not find label for last acq {last_acq} in window {i}. Skipping.")
            continue

        X_windows.append(features)
        y_windows.append(label)

    X_windows_np = np.array(X_windows)
    y_windows_np = np.array(y_windows)

    # Handle case where no windows were created
    if X_windows_np.size == 0:
         return np.array([]), np.array([])

    return X_windows_np, y_windows_np

def visualize_data(raw_data, feature_data, split_data):
    """
    Create visualizations of the data, updated for 4 categories.
    """
    # --- Create visualizations directory ---
    viz_dir = './visualizations'
    os.makedirs(viz_dir, exist_ok=True)

    # --- Check if data exists ---
    if not raw_data:
        print("Cannot visualize: Raw data is empty.")
        return
    freq = list(raw_data.keys())[0] # Use the first frequency for examples
    if freq not in raw_data or raw_data[freq].empty:
        print(f"Cannot visualize: Raw data for {freq} is empty.")
        return

    # --- 1. Plot sample raw signals ---
    sample_df = raw_data[freq]
    plt.figure(figsize=(20, 5)) # Wider figure for 4 plots
    plt.suptitle(f'Raw Signal Examples - {freq}', fontsize=16)

    for cat_code, cat_name in DAMAGE_CATEGORIES.items():
        ax = plt.subplot(1, len(DAMAGE_CATEGORIES), cat_code + 1)
        # *** MODIFICATION START: Use new column name ***
        cat_df_sample = sample_df[sample_df['damage_category'] == cat_code]
        # *** MODIFICATION END ***
        # Plot first N points if data exists
        plot_points = cat_df_sample.head(1000) # Sample first 1000 points of the category

        if not plot_points.empty and plot_points.shape[1] >= 3: # Check columns exist
            # Assuming columns 0=distance/index, 1=torsional, 2=flexural
            ax.plot(plot_points.iloc[:, 0], plot_points.iloc[:, 1], label='Torsional', alpha=0.8)
            ax.plot(plot_points.iloc[:, 0], plot_points.iloc[:, 2], label='Flexural', alpha=0.8)
            ax.set_title(cat_name)
            ax.set_xlabel('Index/Distance')
            if cat_code == 0: # Add Y label only to first plot
                ax.set_ylabel('Amplitude')
            ax.legend()
        else:
            ax.set_title(f"{cat_name}\n(No Data/Fewer Cols)")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(viz_dir, 'raw_signals_comparison_4cat.png'))
    plt.close() # Close the figure


    # --- 2. Plot feature distributions per category ---
    if freq not in feature_data or feature_data[freq].empty:
        print(f"Skipping feature distribution plot for {freq}: No features.")
        return
    feature_df = feature_data[freq]

    plt.figure(figsize=(18, 12))
    plt.suptitle(f'Feature Distributions by Damage Category - {freq}', fontsize=16)
    # *** MODIFICATION START: Use new column name ***
    features_to_plot = [col for col in feature_df.columns if col not in ['acquisition_number', 'damage_category']]
    # *** MODIFICATION END ***
    features_to_plot = features_to_plot[:min(len(features_to_plot), 9)] # Plot max 9

    num_plots = len(features_to_plot)
    if num_plots == 0:
         print(f"No features found to plot distributions for {freq}.")
         plt.close()
         return
    num_cols = 3
    num_rows = (num_plots + num_cols - 1) // num_cols
    colors = plt.cm.viridis(np.linspace(0, 1, len(DAMAGE_CATEGORIES))) # Color map

    for i, feature in enumerate(features_to_plot):
        ax = plt.subplot(num_rows, num_cols, i + 1)
        for cat_code, cat_name in DAMAGE_CATEGORIES.items():
            # *** MODIFICATION START: Use new column name ***
            subset = feature_df[feature_df['damage_category'] == cat_code][feature]
            # *** MODIFICATION END ***
            if not subset.empty:
                # Use seaborn histplot for better density estimation
                sns.histplot(subset, kde=True, label=cat_name, ax=ax, color=colors[cat_code], stat="density", common_norm=False, element="step")
        ax.set_title(f'Distribution of {feature}')
        ax.set_xlabel(feature)
        ax.set_ylabel('Density')
        ax.legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(os.path.join(viz_dir, 'feature_distributions_4cat.png'))
    plt.close()


    # --- 3. Plot class distribution in train/val/test sets ---
    if freq not in split_data or not all(s in split_data[freq] for s in ['train', 'val', 'test']):
         print(f"Skipping split distribution plot for {freq}: Split data incomplete.")
         return

    all_counts = {}
    splits_exist = False
    for name, df in [('Train', split_data[freq]['train']), ('Validation', split_data[freq]['val']), ('Test', split_data[freq]['test'])]:
        # *** MODIFICATION START: Use new column name ***
        if not df.empty and 'damage_category' in df.columns:
            all_counts[name] = df['damage_category'].value_counts()
            splits_exist = True
        else:
            all_counts[name] = pd.Series(dtype=int)
        # *** MODIFICATION END ***

    if not splits_exist:
        print(f"Skipping split distribution plot for {freq}: No valid split data.")
        return

    # Create DataFrame from counts for easier plotting
    count_df = pd.DataFrame(all_counts).fillna(0).astype(int)
    # Ensure all categories (0-3) are present as rows, map index to names
    count_df = count_df.reindex(list(DAMAGE_CATEGORIES.keys()), fill_value=0)
    count_df.index = count_df.index.map(DAMAGE_CATEGORIES) # Use names

    # Plotting
    count_df.plot(kind='bar', figsize=(12, 7), width=0.8)
    plt.ylabel('Count')
    plt.xlabel('Damage Category')
    plt.title(f'Class Distribution in Train/Val/Test Sets - {freq}')
    plt.xticks(rotation=0) # Keep category names horizontal
    plt.legend(title='Dataset Split')
    plt.tight_layout()
    plt.savefig(os.path.join(viz_dir, 'class_distribution_split_4cat.png'))
    plt.close()


def main():
    """
    Main function to run the data preparation pipeline.
    """
    # Define file paths
    file_path = "/content/Merged_Pipeline_Data.xlsx" # Make sure this path is correct

    # Select which frequency sheets to process
    selected_sheets = ["Frequency_1", "Frequency_2","Frequency_3", "Frequency_4","Frequency_5"] # Example

    # Define acquisition ranges (USE THE NUMBERS FROM YOUR DATA)
    # Example using the numbers from your previous successful run's output
    acquisition_info = {
        'start': 22,           # Corresponds to original #1622?
        'end': 330,            # Corresponds to original #1930?
        'damage_start': 302,   # Corresponds to original #1902?
        'total_acquisitions': 309 # end - start + 1
    }
    # OR if using original numbers directly:
    # acquisition_info = {
    #     'start': 1622,
    #     'end': 1930,
    #     'damage_start': 1902,
    #     'total_acquisitions': 1930 - 1622 + 1 # Calculate total
    # }
    # Choose ONE version of acquisition_info based on how your data is structured

    # Create output directories
    proc_dir = './processed_data'
    viz_dir = './visualizations'
    os.makedirs(proc_dir, exist_ok=True)
    os.makedirs(viz_dir, exist_ok=True) # Also ensure viz dir exists

    # 1. Load data
    print("--- Loading Data ---")
    raw_data = load_frequency_data(file_path, selected_sheets)
    if raw_data is None: return # Exit if loading fails

    # 2. Add acquisition information and damage CATEGORY labels (0-3)
    print("\n--- Adding Acquisition Info & Damage Categories ---")
    labeled_data = add_acquisition_info(raw_data, acquisition_info)

    # 3. Extract features
    print("\n--- Extracting Features ---")
    feature_data = extract_features(labeled_data)

    # 4. Normalize features
    print("\n--- Normalizing Features ---")
    normalized_data, scalers = normalize_features(feature_data)

    # 5. Split data
    print("\n--- Splitting Data ---")
    split_data_dict = split_data(normalized_data, test_size=0.2, validation_size=0.15) # Pass validation_size

    # 6. Create time windows
    print("\n--- Creating Time Windows ---")
    windowed_data = create_time_windows(split_data_dict, window_size=5, step=1) # Pass step

    # 7. Visualize the data
    print("\n--- Creating Visualizations ---")
    visualize_data(labeled_data, feature_data, split_data_dict)

    # 8. Save processed data
    print("\n--- Saving Processed Data ---")
    for freq, data_dict in split_data_dict.items():
        # Save split features (CSV) - keeping acquisition number is useful
        if not data_dict['train'].empty:
            data_dict['train'].to_csv(f'{proc_dir}/{freq}_train_features.csv', index=False)
        if not data_dict['val'].empty:
            data_dict['val'].to_csv(f'{proc_dir}/{freq}_val_features.csv', index=False)
        if not data_dict['test'].empty:
            data_dict['test'].to_csv(f'{proc_dir}/{freq}_test_features.csv', index=False)

    for freq, window_dict in windowed_data.items():
        # Save windowed features (NPY) - X includes features, y includes category
        # Check if arrays are not empty before saving
        if window_dict.get('train') and window_dict['train'][0].size > 0:
            np.save(f'{proc_dir}/{freq}_train_windows_X.npy', window_dict['train'][0])
            np.save(f'{proc_dir}/{freq}_train_windows_y.npy', window_dict['train'][1])
        if window_dict.get('val') and window_dict['val'][0].size > 0:
            np.save(f'{proc_dir}/{freq}_val_windows_X.npy', window_dict['val'][0])
            np.save(f'{proc_dir}/{freq}_val_windows_y.npy', window_dict['val'][1])
        if window_dict.get('test') and window_dict['test'][0].size > 0:
            np.save(f'{proc_dir}/{freq}_test_windows_X.npy', window_dict['test'][0])
            np.save(f'{proc_dir}/{freq}_test_windows_y.npy', window_dict['test'][1])

    print("\n--- Data Preparation Complete! ---")
    print(f"Saved processed features (CSV) and windowed data (NPY) to '{proc_dir}'")
    print(f"Saved visualizations to '{viz_dir}'")


if __name__ == "__main__":
    main()

--- Loading Data ---
Loaded Frequency_1 data: 591652 rows, 4 columns
Loaded Frequency_2 data: 591652 rows, 4 columns
Loaded Frequency_3 data: 591652 rows, 4 columns
Loaded Frequency_4 data: 591652 rows, 4 columns
Loaded Frequency_5 data: 591652 rows, 4 columns

--- Adding Acquisition Info & Damage Categories ---
Categorization Ranges: Normal(<302), Micro(302-310), Minor(311-319), Major(>319)
Estimating ~1915 measurements per acquisition for Frequency_1.
 -> Added info for Frequency_1. Category distribution: damage_category
0    536200
1     17235
2     17235
3     20982
Name: count, dtype: int64
Estimating ~1915 measurements per acquisition for Frequency_2.
 -> Added info for Frequency_2. Category distribution: damage_category
0    536200
1     17235
2     17235
3     20982
Name: count, dtype: int64
Estimating ~1915 measurements per acquisition for Frequency_3.
 -> Added info for Frequency_3. Category distribution: damage_category
0    536200
1     17235
2     17235
3     20982
Name: c

In [2]:
!pip install xgboost==1.7.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.8 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4


In [6]:
pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 11.5 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=3caf5723e200e3372b4d60cdc3e648da3c31e2579f4e59f9a55d08b02e05a0bb
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # Added
import os
import joblib
import warnings

# Scikit-learn imports
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest, RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, mean_absolute_error, confusion_matrix, accuracy_score, roc_auc_score, mean_squared_error, r2_score
from sklearn.svm import OneClassSVM

# TensorFlow / Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# XGBoost import
import xgboost as xgb

# +++ Lifelines Import +++
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
# +++ End Lifelines Import +++

In [17]:
warnings.filterwarnings('ignore')

# Define directories
MODEL_SAVE_DIR = './saved_models/'
SCALER_SAVE_DIR = './saved_scalers/'
VIZ_SAVE_DIR = './visualizations/' # Added
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(SCALER_SAVE_DIR, exist_ok=True)
os.makedirs(VIZ_SAVE_DIR, exist_ok=True) # Added

class PipelineCrackAnalysisPipeline:
    """
    Updated pipeline including visualization and basic survival analysis.
    Assumes pre-processed data available:
    - Window sequences: *_windows_X.npy
    - Window labels (0-3): *_windows_y.npy
    - Extracted features (with acquisition_number): *_features.csv
    """
    def __init__(self, data_dir='./processed_data/', frequencies=['Frequency_1', 'Frequency_2']):
        self.data_dir = data_dir
        self.frequencies = frequencies
        self.data = {}
        self.models = {
            'anomaly_detector': {},
            'crack_classifier': {},
            'lifetime_predictor': {} # Will store RUL regressors AND Cox model
        }
        self.scalers = {}
        self.feature_importances = {}
        self.evaluation_results = {
             'anomaly_detection': {},
             'crack_classification': {},
             'lifetime_prediction': {} # Will store RUL and Survival results
        }
        self.crack_categories = {0: 'Normal', 1: 'Micro', 2: 'Minor', 3: 'Major'}


    def load_data(self):
        """
        Load processed windowed data (NPY) and extracted feature data (CSV).
        Assumes _windows_y.npy and CSV 'damage_category' contain labels 0-3.
        Assumes *_features.csv contains 'acquisition_number'.
        """
        # --- This function remains the same ---
        print("--- Loading Data ---")
        self.data = {freq: {} for freq in self.frequencies}
        found_all_files = True
        for freq in self.frequencies:
            print(f"Loading data for {freq}...")
            try:
                self.data[freq]['train_windows_X'] = np.load(f'{self.data_dir}/{freq}_train_windows_X.npy', allow_pickle=True)
                self.data[freq]['train_windows_y'] = np.load(f'{self.data_dir}/{freq}_train_windows_y.npy', allow_pickle=True)
                self.data[freq]['val_windows_X'] = np.load(f'{self.data_dir}/{freq}_val_windows_X.npy', allow_pickle=True)
                self.data[freq]['val_windows_y'] = np.load(f'{self.data_dir}/{freq}_val_windows_y.npy', allow_pickle=True)
                self.data[freq]['test_windows_X'] = np.load(f'{self.data_dir}/{freq}_test_windows_X.npy', allow_pickle=True)
                self.data[freq]['test_windows_y'] = np.load(f'{self.data_dir}/{freq}_test_windows_y.npy', allow_pickle=True)
                print(" -> Loaded windowed NPY data.")
                self.data[freq]['train_features_df'] = pd.read_csv(f'{self.data_dir}/{freq}_train_features.csv')
                self.data[freq]['val_features_df'] = pd.read_csv(f'{self.data_dir}/{freq}_val_features.csv')
                self.data[freq]['test_features_df'] = pd.read_csv(f'{self.data_dir}/{freq}_test_features.csv')
                print(" -> Loaded feature CSV data.")
                for split in ['train', 'val', 'test']:
                     df_key = f'{split}_features_df'; required_cols = {'acquisition_number', 'damage_category'}
                     if df_key in self.data[freq] and not required_cols.issubset(self.data[freq][df_key].columns):
                          raise ValueError(f"Missing required columns in {df_key} for {freq}")
                print(f" -> Windowed data shapes (Train X/y, Val X/y, Test X/y):")
                print(f"    {self.data[freq]['train_windows_X'].shape}, {self.data[freq]['train_windows_y'].shape}")
                # ... [rest of print statements as before] ...
            except FileNotFoundError as e: print(f"Error: {e}"); found_all_files=False; raise
            except Exception as e: print(f"Error: {e}"); found_all_files=False; raise
        if found_all_files: print("Data loaded successfully.")
        else: print("Data loading incomplete.")

    def _scale_windowed_data(self, freq):
        """Scales the windowed feature data (X) using StandardScaler."""
        # --- Function code filled in (same as before) ---
        print(f"Scaling windowed data for {freq}...")
        scaler = StandardScaler()
        if self.data[freq]['train_windows_X'].size == 0:
             print(f" -> Skipping scaling for {freq}: Train X data is empty.")
             self.scalers[f"{freq}_windows"] = None
             self.data[freq]['train_windows_X_scaled'] = np.array([])
             self.data[freq]['val_windows_X_scaled'] = np.array([])
             self.data[freq]['test_windows_X_scaled'] = np.array([])
             return
        train_shape = self.data[freq]['train_windows_X'].shape
        val_shape = self.data[freq]['val_windows_X'].shape
        test_shape = self.data[freq]['test_windows_X'].shape
        try:
            if len(train_shape) < 3 or train_shape[-1] == 0: raise ValueError(f"Invalid shape: {train_shape}")
            train_reshaped = self.data[freq]['train_windows_X'].reshape(-1, train_shape[-1])
            scaler.fit(train_reshaped)
            self.data[freq]['train_windows_X_scaled'] = scaler.transform(train_reshaped).reshape(train_shape)
            if self.data[freq]['val_windows_X'].size > 0:
                 if len(val_shape) < 3 or val_shape[-1] != train_shape[-1]: raise ValueError(f"Val X shape mismatch: {val_shape}")
                 val_reshaped = self.data[freq]['val_windows_X'].reshape(-1, val_shape[-1])
                 self.data[freq]['val_windows_X_scaled'] = scaler.transform(val_reshaped).reshape(val_shape)
            else: self.data[freq]['val_windows_X_scaled'] = np.array([])
            if self.data[freq]['test_windows_X'].size > 0:
                 if len(test_shape) < 3 or test_shape[-1] != train_shape[-1]: raise ValueError(f"Test X shape mismatch: {test_shape}")
                 test_reshaped = self.data[freq]['test_windows_X'].reshape(-1, test_shape[-1])
                 self.data[freq]['test_windows_X_scaled'] = scaler.transform(test_reshaped).reshape(test_shape)
            else: self.data[freq]['test_windows_X_scaled'] = np.array([])
            self.scalers[f"{freq}_windows"] = scaler
            print(f" -> Windowed data scaled for {freq}.")
        except ValueError as e:
             print(f"Error scaling windowed data for {freq}: {e}")
             self.scalers[f"{freq}_windows"] = None
             self.data[freq]['train_windows_X_scaled'] = np.array([])
             self.data[freq]['val_windows_X_scaled'] = np.array([])
             self.data[freq]['test_windows_X_scaled'] = np.array([])

    def _determine_threshold(self, model, data_normal, strategy='percentile', percentile=95):
        """
        Helper to determine anomaly threshold based on scores/errors from normal data.
        Args:
            model: The fitted anomaly detection model.
            data_normal: Normal data (NumPy array or Pandas DataFrame).
            strategy (str): Method ('percentile').
            percentile (int): Percentile value.
        Returns: float: Anomaly threshold.
        """
        if isinstance(data_normal, pd.DataFrame) and data_normal.empty: data_normal = np.array([])
        elif not isinstance(data_normal, (np.ndarray, pd.DataFrame)): raise TypeError("data_normal must be NumPy array or Pandas DataFrame")
        if data_normal.size == 0: print("Warning: Cannot determine threshold from empty normal data."); return np.inf

        print(f"Determining threshold using {model.__class__.__name__} on data shape {data_normal.shape}")
        data_normal_np = data_normal.values if isinstance(data_normal, pd.DataFrame) else data_normal

        # --- Determine expected data dimensionality based on model type ---
        if model.__class__.__name__ in ['Functional', 'Sequential']: # Keras model
            is_lstm_ae = False
            if model.__class__.__name__ == 'Sequential' and len(model.layers) > 0 and isinstance(model.layers[0], LSTM):
                 is_lstm_ae = True # Assuming LSTM AE is Sequential starting with LSTM

            expected_data_dims = 3 if is_lstm_ae else 2

            # --- CORRECTED Dimension Check ---
            if len(data_normal_np.shape) != expected_data_dims:
                 raise ValueError(f"Input data dimension mismatch for Keras model {model.name}. Expected {expected_data_dims}D, got {len(data_normal_np.shape)}D (shape {data_normal_np.shape}).")

            # Calculate reconstruction error
            reconstructions = model.predict(data_normal_np, verbose=0)
            if expected_data_dims == 3: # LSTM AE
                errors = np.mean(np.mean(np.square(data_normal_np - reconstructions), axis=2), axis=1)
            else: # Dense AE
                errors = np.mean(np.square(data_normal_np - reconstructions), axis=1)

        elif model.__class__.__name__ == 'IsolationForest':
             errors = -model.decision_function(data_normal_np)
        elif model.__class__.__name__ == 'OneClassSVM':
              errors = -model.decision_function(data_normal_np)
        else:
            raise ValueError(f"Unsupported model type '{model.__class__.__name__}' for threshold determination.")

        # --- Calculate Threshold ---
        if strategy == 'percentile':
            if errors.size == 0: print("Warning: No errors calculated."); return np.inf
            if np.all(errors == errors[0]): print(f"Warning: All errors identical ({errors[0]:.4f})."); return errors[0] # Return the value itself
            return np.percentile(errors, percentile)
        else:
             raise ValueError(f"Unsupported thresholding strategy: {strategy}")

    def _extract_signal_features(self, freq):
        """Creates scaled feature sets from loaded CSVs."""
        # --- Function code filled in (same as before) ---
        print(f"Creating scaled feature set for {freq}...")
        for dataset in ['train', 'val', 'test']:
             df_key = f'{dataset}_features_df'; scaled_df_key = f'{dataset}_features_scaled'
             if df_key not in self.data[freq] or self.data[freq][df_key].empty:
                 print(f" -> Skipping feature scaling for {freq}-{dataset}: Source CSV missing or empty.")
                 self.data[freq][scaled_df_key] = pd.DataFrame()
                 if dataset == 'train': self.scalers[f"{freq}_features"] = None
                 continue
             df_unscaled = self.data[freq][df_key]
             ids_labels = df_unscaled[['acquisition_number', 'damage_category']]
             features_unscaled = df_unscaled.drop(['acquisition_number', 'damage_category'], axis=1)
             if features_unscaled.empty:
                  print(f" -> No features to scale for {freq}-{dataset}.")
                  self.data[freq][scaled_df_key] = ids_labels.copy()
                  if dataset == 'train': self.scalers[f"{freq}_features"] = None
                  continue
             if dataset == 'train':
                  try:
                       feature_scaler = StandardScaler(); feature_scaler.fit(features_unscaled)
                       self.scalers[f"{freq}_features"] = feature_scaler
                       print(f" -> Fitted feature scaler for {freq} on training data.")
                  except ValueError as e: print(f"Error fitting feature scaler for {freq}: {e}. Skipping."); self.scalers[f"{freq}_features"] = None; break
             scaler = self.scalers.get(f"{freq}_features")
             if scaler is None:
                  print(f" -> Skipping scaling transform for {freq}-{dataset}: Scaler not available.")
                  self.data[freq][scaled_df_key] = df_unscaled.copy(); continue
             try:
                 scaled_features_array = scaler.transform(features_unscaled)
                 scaled_features_df = pd.DataFrame(scaled_features_array, columns=features_unscaled.columns, index=features_unscaled.index)
                 self.data[freq][scaled_df_key] = pd.concat([ids_labels, scaled_features_df], axis=1)
                 print(f" -> Scaled features created for {freq}-{dataset}.")
             except ValueError as e: print(f"Error transforming features for {freq}-{dataset}: {e}. Using unscaled."); self.data[freq][scaled_df_key] = df_unscaled.copy()
        print(f" -> Feature scaling preparation done for {freq}.")

    def preprocess_features(self):
        """Orchestrates preprocessing."""
        # --- Same as previous version ---
        print("\n--- Feature Preprocessing ---")
        for freq in self.frequencies:
            self._extract_signal_features(freq)
            self._scale_windowed_data(freq)
        print("Feature preprocessing completed for all frequencies.")

    def build_anomaly_detector(self):
        """Build anomaly detection models (Stage 1)."""
        print("\n--- Building Anomaly Detection Models (Stage 1) ---")
        for freq in self.frequencies:
             print(f"\nTraining anomaly detectors for {freq}...")
             # Check data existence
             if 'train_features_scaled' not in self.data[freq] or self.data[freq]['train_features_scaled'].empty:
                 print(f" -> Skipping anomaly detection for {freq}: Missing or empty scaled features.")
                 continue

             # Prepare normal data
             train_features_normal = self.data[freq]['train_features_scaled'][
                 self.data[freq]['train_features_scaled']['damage_category'] == 0
             ].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore') # Drop all non-feature cols

             if 'train_windows_X_scaled' not in self.data[freq] or self.data[freq]['train_windows_X_scaled'].size == 0:
                 train_windows_normal_scaled = np.array([])
                 print(f"Warning: Scaled window data missing/empty for {freq}. LSTM AE cannot be trained.")
             else:
                 train_windows_normal_scaled = self.data[freq]['train_windows_X_scaled'][
                     self.data[freq]['train_windows_y'] == 0 ]

             # --- Train Feature-Based Anomaly Detectors ---
             if not train_features_normal.empty:
                 # 1. Isolation Forest
                 print("Training Isolation Forest..."); iso_forest = IsolationForest(n_estimators=150, contamination='auto', max_samples='auto', random_state=42, n_jobs=-1)
                 iso_forest.fit(train_features_normal); self.models['anomaly_detector'][f'{freq}_isolation_forest'] = iso_forest; print(" -> IF trained.")
                 # 2. One-Class SVM
                 print("Training One-Class SVM..."); ocsvm = OneClassSVM(nu=0.05, kernel='rbf', gamma='scale')
                 ocsvm.fit(train_features_normal); self.models['anomaly_detector'][f'{freq}_ocsvm'] = ocsvm; print(" -> OCSVM trained.")
                 # 3. Dense Autoencoder
                 print("Training Dense Autoencoder..."); input_dim = train_features_normal.shape[1]
                 if input_dim > 0:
                      encoding_dim = max(8, input_dim // 8); input_layer = Input(shape=(input_dim,)); encoded = Dense(input_dim // 2, activation='relu')(input_layer); encoded = Dropout(0.1)(encoded); encoded = Dense(input_dim // 4, activation='relu')(encoded); encoded = Dense(encoding_dim, activation='relu')(encoded); decoded = Dense(input_dim // 4, activation='relu')(encoded); decoded = Dropout(0.1)(decoded); decoded = Dense(input_dim // 2, activation='relu')(decoded); decoded = Dense(input_dim, activation='linear')(decoded); autoencoder = Model(input_layer, decoded)
                      autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
                      # Prep validation data
                      val_features_normal = self.data[freq].get('val_features_scaled', pd.DataFrame())
                      if not val_features_normal.empty: val_features_normal = val_features_normal[val_features_normal['damage_category'] == 0].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
                      val_data_ae = (val_features_normal, val_features_normal) if not val_features_normal.empty else None
                      callbacks_ae = [EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, mode='min')] if val_data_ae else None
                      autoencoder.fit(train_features_normal, train_features_normal, epochs=150, batch_size=64, shuffle=True, validation_data=val_data_ae, callbacks=callbacks_ae, verbose=0)
                      self.models['anomaly_detector'][f'{freq}_autoencoder'] = autoencoder; print(" -> Dense AE trained.")
                 else: print(" -> Skipping Dense AE: No features.")
             else: print("Skipping feature-based AD due to no normal feature data.")

             # --- Train Sequence-Based Anomaly Detector ---
             if train_windows_normal_scaled.size > 0:
                 print("Training LSTM Autoencoder..."); timesteps, features = train_windows_normal_scaled.shape[1:]
                 if features > 0:
                      lstm_encoding_dim = max(16, features // 2)
                      lstm_autoencoder = Sequential([ LSTM(128, activation='relu', input_shape=(timesteps, features), return_sequences=True), Dropout(0.2), LSTM(lstm_encoding_dim, activation='relu', return_sequences=False), RepeatVector(timesteps), LSTM(lstm_encoding_dim, activation='relu', return_sequences=True), Dropout(0.2), LSTM(128, activation='relu', return_sequences=True), TimeDistributed(Dense(features, activation='linear')) ])
                      lstm_autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
                      # Prep validation data
                      val_windows_normal_scaled = np.array([])
                      if 'val_windows_X_scaled' in self.data[freq] and self.data[freq]['val_windows_X_scaled'].size > 0:
                           val_windows_normal_scaled = self.data[freq]['val_windows_X_scaled'][self.data[freq]['val_windows_y'] == 0]
                      val_data_lstm = (val_windows_normal_scaled, val_windows_normal_scaled) if val_windows_normal_scaled.size > 0 else None
                      callbacks_lstm = [EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, mode='min')] if val_data_lstm else None
                      lstm_autoencoder.fit(train_windows_normal_scaled, train_windows_normal_scaled, epochs=75, batch_size=64, shuffle=True, validation_data=val_data_lstm, callbacks=callbacks_lstm, verbose=0)
                      self.models['anomaly_detector'][f'{freq}_lstm_autoencoder'] = lstm_autoencoder; print(" -> LSTM AE trained.")
                 else: print(" -> Skipping LSTM AE: No features in windowed data.")
             else: print("Skipping LSTM AE due to no normal window data.")
        print("\nAnomaly detection model building completed.")


    def evaluate_anomaly_detector(self, threshold_percentile=95):
        """Evaluate anomaly detection models (Stage 1)."""
        print("\n--- Evaluating Anomaly Detection Models ---")
        results = {}; anomaly_thresholds = {}
        for freq in self.frequencies:
            print(f"\nEvaluating anomaly detectors for {freq}..."); results[freq] = {}; anomaly_thresholds[freq] = {}
            # Check data
            if 'test_features_scaled' not in self.data[freq] or self.data[freq]['test_features_scaled'].empty:
                print(f" -> Skipping anomaly evaluation for {freq}: Test features missing/empty."); continue
            test_features = self.data[freq]['test_features_scaled'].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
            true_labels_features = (self.data[freq]['test_features_scaled']['damage_category'] > 0).astype(int)
            if 'test_windows_X_scaled' in self.data[freq] and self.data[freq]['test_windows_X_scaled'].size > 0:
                 test_windows_scaled = self.data[freq]['test_windows_X_scaled']
                 true_labels_windows = (self.data[freq]['test_windows_y'] > 0).astype(int)
            else: test_windows_scaled = np.array([]); true_labels_windows = np.array([])

            # Get normal validation data for thresholding
            val_features_normal = self.data[freq].get('val_features_scaled', pd.DataFrame())
            if not val_features_normal.empty: val_features_normal = val_features_normal[val_features_normal['damage_category'] == 0].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
            val_windows_normal_scaled = np.array([])
            if 'val_windows_X_scaled' in self.data[freq] and self.data[freq]['val_windows_X_scaled'].size > 0:
                 val_windows_normal_scaled = self.data[freq]['val_windows_X_scaled'][self.data[freq]['val_windows_y'] == 0]

            # Evaluate IF
            if f'{freq}_isolation_forest' in self.models['anomaly_detector']:
                print("Evaluating Isolation Forest..."); iso_forest = self.models['anomaly_detector'][f'{freq}_isolation_forest']
                iso_scores = iso_forest.decision_function(test_features); iso_predictions = (iso_forest.predict(test_features) == -1).astype(int)
                try: auc = roc_auc_score(true_labels_features, -iso_scores)
                except ValueError: auc = float('nan')
                results[freq]['Isolation Forest'] = {'Accuracy': accuracy_score(true_labels_features, iso_predictions), 'AUC': auc, 'Confusion Matrix': confusion_matrix(true_labels_features, iso_predictions).tolist(), 'Classification Report': classification_report(true_labels_features, iso_predictions, output_dict=True, zero_division=0)}
                print(f" -> IF Accuracy: {results[freq]['Isolation Forest']['Accuracy']:.4f}, AUC: {results[freq]['Isolation Forest']['AUC']:.4f}")
            # Evaluate OCSVM
            if f'{freq}_ocsvm' in self.models['anomaly_detector']:
                 print("Evaluating One-Class SVM..."); ocsvm = self.models['anomaly_detector'][f'{freq}_ocsvm']
                 ocsvm_scores = ocsvm.decision_function(test_features); ocsvm_predictions = (ocsvm.predict(test_features) == -1).astype(int)
                 try: auc = roc_auc_score(true_labels_features, -ocsvm_scores)
                 except ValueError: auc = float('nan')
                 results[freq]['One-Class SVM'] = {'Accuracy': accuracy_score(true_labels_features, ocsvm_predictions), 'AUC': auc, 'Confusion Matrix': confusion_matrix(true_labels_features, ocsvm_predictions).tolist(), 'Classification Report': classification_report(true_labels_features, ocsvm_predictions, output_dict=True, zero_division=0)}
                 print(f" -> OCSVM Accuracy: {results[freq]['One-Class SVM']['Accuracy']:.4f}, AUC: {results[freq]['One-Class SVM']['AUC']:.4f}")
            # Evaluate Dense AE
            if f'{freq}_autoencoder' in self.models['anomaly_detector']:
                 print("Evaluating Dense Autoencoder..."); autoencoder = self.models['anomaly_detector'][f'{freq}_autoencoder']
                 ae_threshold = np.inf # Default if no normal data
                 if not val_features_normal.empty: ae_threshold = self._determine_threshold(autoencoder, val_features_normal, percentile=threshold_percentile)
                 else: print(" -> Warning: Cannot determine AE threshold.");
                 anomaly_thresholds[freq]['dense_autoencoder'] = ae_threshold
                 ae_reconstructions = autoencoder.predict(test_features, verbose=0)
                 ae_mse = np.mean(np.square(test_features.values - ae_reconstructions), axis=1)
                 ae_predictions = (ae_mse > ae_threshold).astype(int)
                 try: auc = roc_auc_score(true_labels_features, ae_mse)
                 except ValueError: auc = float('nan')
                 results[freq]['Dense Autoencoder'] = {'Accuracy': accuracy_score(true_labels_features, ae_predictions), 'AUC': auc, 'Threshold': ae_threshold, 'Confusion Matrix': confusion_matrix(true_labels_features, ae_predictions).tolist(), 'Classification Report': classification_report(true_labels_features, ae_predictions, output_dict=True, zero_division=0), 'reconstruction_error': ae_mse} # Added errors
                 print(f" -> Dense AE Accuracy: {results[freq]['Dense Autoencoder']['Accuracy']:.4f}, AUC: {results[freq]['Dense Autoencoder']['AUC']:.4f} (Thresh: {ae_threshold:.4f})")
            # Evaluate LSTM AE
            if f'{freq}_lstm_autoencoder' in self.models['anomaly_detector']:
                 print("Evaluating LSTM Autoencoder...")
                 lstm_ae = self.models['anomaly_detector'][f'{freq}_lstm_autoencoder']
                 lstm_threshold = np.inf
                 if val_windows_normal_scaled.size > 0: lstm_threshold = self._determine_threshold(lstm_ae, val_windows_normal_scaled, percentile=threshold_percentile)
                 else: print(" -> Warning: Cannot determine LSTM AE threshold.")
                 anomaly_thresholds[freq]['lstm_autoencoder'] = lstm_threshold
                 if test_windows_scaled.size > 0:
                      lstm_reconstructions = lstm_ae.predict(test_windows_scaled, verbose=0)
                      lstm_mse = np.mean(np.mean(np.square(test_windows_scaled - lstm_reconstructions), axis=2), axis=1)
                      lstm_predictions = (lstm_mse > lstm_threshold).astype(int)
                      try: auc = roc_auc_score(true_labels_windows, lstm_mse)
                      except ValueError: auc = float('nan')
                      results[freq]['LSTM Autoencoder'] = {'Accuracy': accuracy_score(true_labels_windows, lstm_predictions), 'AUC': auc, 'Threshold': lstm_threshold, 'Confusion Matrix': confusion_matrix(true_labels_windows, lstm_predictions).tolist(), 'Classification Report': classification_report(true_labels_windows, lstm_predictions, output_dict=True, zero_division=0), 'reconstruction_error': lstm_mse} # Added errors
                      print(f" -> LSTM AE Accuracy: {results[freq]['LSTM Autoencoder']['Accuracy']:.4f}, AUC: {results[freq]['LSTM Autoencoder']['AUC']:.4f} (Thresh: {lstm_threshold:.4f})")
                 else: print(" -> Skipping LSTM AE eval: No test window data.")

        self.evaluation_results['anomaly_detection'] = results
        self.evaluation_results['anomaly_thresholds'] = anomaly_thresholds
        print("\nAnomaly detection evaluation completed.")
        return results


    def build_crack_classifier(self):
        """Build classification models (Stage 2) using labels 0-3."""
        # --- Function code filled in (same as previous correct 4-cat version) ---
        print("\n--- Building Crack Classification Models (Stage 2) ---")
        for freq in self.frequencies:
            print(f"\nTraining crack classifiers for {freq}...")
            if 'train_features_scaled' not in self.data[freq] or self.data[freq]['train_features_scaled'].empty:
                 print(f" -> Skipping classifier training for {freq}: Features missing/empty."); continue
            X_train = self.data[freq]['train_features_scaled'].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
            y_train_cat = self.data[freq]['train_features_scaled']['damage_category'].values
            X_val = pd.DataFrame(); y_val_cat = np.array([]); val_set_for_xgb = None; val_data_for_nn = None
            if 'val_features_scaled' in self.data[freq] and not self.data[freq]['val_features_scaled'].empty:
                 X_val = self.data[freq]['val_features_scaled'].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
                 y_val_cat = self.data[freq]['val_features_scaled']['damage_category'].values
                 val_set_for_xgb = [(X_val, y_val_cat)] if not X_val.empty else None
                 val_data_for_nn = (X_val, y_val_cat) if not X_val.empty else None
            else: print(f" -> Warning: Validation data missing/empty for {freq}.")
            num_classes = len(self.crack_categories)
            print(f"Training label distribution: {np.bincount(y_train_cat, minlength=num_classes)}")
            if y_val_cat.size > 0: print(f"Validation label distribution: {np.bincount(y_val_cat, minlength=num_classes)}")
            unique_train_labels = np.unique(y_train_cat); print(f"Unique training labels found: {unique_train_labels}")
            if len(unique_train_labels) < num_classes: print(f"Warning: Training data only contains {len(unique_train_labels)}/{num_classes} categories.")
            if X_train.empty: print(" -> Skipping: Training data is empty."); continue

            # Train RF
            print("Training Random Forest Classifier..."); rf_classifier = RandomForestClassifier(n_estimators=200, max_depth=15, min_samples_split=5, min_samples_leaf=3, class_weight='balanced', random_state=42, n_jobs=-1)
            rf_classifier.fit(X_train, y_train_cat); self.models['crack_classifier'][f'{freq}_random_forest'] = rf_classifier
            if not X_val.empty: val_preds_rf = rf_classifier.predict(X_val); val_acc_rf = accuracy_score(y_val_cat, val_preds_rf); print(f" -> RF Validation Accuracy: {val_acc_rf:.4f}")
            else: print(" -> RF Validation skipped.");
            if hasattr(X_train, 'columns'): self.feature_importances[f'{freq}_crack_classification_rf'] = pd.Series(rf_classifier.feature_importances_, index=X_train.columns).sort_values(ascending=False)

            # Train XGBoost
            print("Training XGBoost Classifier..."); xgb_classifier = xgb.XGBClassifier(n_estimators=150, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8, objective='multi:softmax', num_class=num_classes, random_state=42, n_jobs=-1)
            try:
                 xgb_classifier.fit(X_train, y_train_cat, eval_set=val_set_for_xgb, eval_metric='mlogloss', early_stopping_rounds=10 if val_set_for_xgb else None, verbose=False)
                 self.models['crack_classifier'][f'{freq}_xgboost'] = xgb_classifier
                 if val_set_for_xgb: val_preds_xgb = xgb_classifier.predict(X_val); val_acc_xgb = accuracy_score(y_val_cat, val_preds_xgb); print(f" -> XGB Validation Accuracy: {val_acc_xgb:.4f}")
                 else: print(" -> XGB Validation skipped.")
            except ValueError as e: print(f"Error training XGBoost for {freq}: {e}")

            # Train NN
            print("Training Dense Neural Network Classifier..."); input_dim = X_train.shape[1]
            if input_dim > 0:
                 nn_classifier = Sequential([ Dense(128, activation='relu', input_shape=(input_dim,)), Dropout(0.4), Dense(64, activation='relu'), Dropout(0.3), Dense(32, activation='relu'), Dense(num_classes, activation='softmax') ])
                 nn_classifier.compile(optimizer=Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
                 callbacks_nn = [EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True, mode='max')] if val_data_for_nn else None
                 nn_classifier.fit(X_train, y_train_cat, epochs=150, batch_size=64, validation_data=val_data_for_nn, callbacks=callbacks_nn, verbose=0)
                 self.models['crack_classifier'][f'{freq}_neural_network'] = nn_classifier
                 if val_data_for_nn: val_loss_nn, val_acc_nn = nn_classifier.evaluate(X_val, y_val_cat, verbose=0); print(f" -> NN Validation Accuracy: {val_acc_nn:.4f}")
                 else: print(" -> NN Validation skipped.")
            else: print(" -> Skipping NN: No input features.")
        print("\nCrack classification model building completed.")


    def evaluate_crack_classifier(self):
        """Evaluate classification models (Stage 2) using labels 0-3."""
        # --- Function code filled in (same as previous correct 4-cat version) ---
        print("\n--- Evaluating Crack Classification Models ---")
        results = {}
        for freq in self.frequencies:
            print(f"\nEvaluating crack classifiers for {freq}..."); results[freq] = {}
            if 'test_features_scaled' not in self.data[freq] or self.data[freq]['test_features_scaled'].empty:
                 print(f" -> Skipping classifier evaluation for {freq}: Test features missing/empty."); continue
            X_test = self.data[freq]['test_features_scaled'].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
            y_test_cat = self.data[freq]['test_features_scaled']['damage_category'].values
            target_names = [self.crack_categories[i] for i in sorted(self.crack_categories.keys())]
            labels_present = sorted(np.unique(y_test_cat)); target_names_present = [self.crack_categories[i] for i in labels_present]
            print(f"Test label distribution: {np.bincount(y_test_cat, minlength=len(self.crack_categories))}")
            if X_test.empty: print(" -> Skipping eval: Test data features are empty."); continue

            # Evaluate RF
            if f'{freq}_random_forest' in self.models['crack_classifier']:
                 print("Evaluating Random Forest Classifier..."); rf_classifier = self.models['crack_classifier'][f'{freq}_random_forest']
                 rf_preds = rf_classifier.predict(X_test); results[freq]['Random Forest'] = {'Accuracy': accuracy_score(y_test_cat, rf_preds), 'Confusion Matrix': confusion_matrix(y_test_cat, rf_preds, labels=labels_present).tolist(), 'Confusion Matrix Labels': labels_present, 'Classification Report': classification_report(y_test_cat, rf_preds, labels=labels_present, target_names=target_names_present, output_dict=True, zero_division=0)}
                 print(f" -> RF Test Accuracy: {results[freq]['Random Forest']['Accuracy']:.4f}")
            # Evaluate XGB
            if f'{freq}_xgboost' in self.models['crack_classifier']:
                 print("Evaluating XGBoost Classifier..."); xgb_classifier = self.models['crack_classifier'][f'{freq}_xgboost']
                 xgb_preds = xgb_classifier.predict(X_test); results[freq]['XGBoost'] = {'Accuracy': accuracy_score(y_test_cat, xgb_preds), 'Confusion Matrix': confusion_matrix(y_test_cat, xgb_preds, labels=labels_present).tolist(), 'Confusion Matrix Labels': labels_present, 'Classification Report': classification_report(y_test_cat, xgb_preds, labels=labels_present, target_names=target_names_present, output_dict=True, zero_division=0)}
                 print(f" -> XGB Test Accuracy: {results[freq]['XGBoost']['Accuracy']:.4f}")
            # Evaluate NN
            if f'{freq}_neural_network' in self.models['crack_classifier']:
                 print("Evaluating Neural Network Classifier..."); nn_classifier = self.models['crack_classifier'][f'{freq}_neural_network']
                 nn_pred_probs = nn_classifier.predict(X_test, verbose=0); nn_preds = np.argmax(nn_pred_probs, axis=1)
                 results[freq]['Neural Network'] = {'Accuracy': accuracy_score(y_test_cat, nn_preds), 'Confusion Matrix': confusion_matrix(y_test_cat, nn_preds, labels=labels_present).tolist(), 'Confusion Matrix Labels': labels_present, 'Classification Report': classification_report(y_test_cat, nn_preds, labels=labels_present, target_names=target_names_present, output_dict=True, zero_division=0)}
                 print(f" -> NN Test Accuracy: {results[freq]['Neural Network']['Accuracy']:.4f}")
        self.evaluation_results['crack_classification'] = results
        print("\nCrack classification evaluation completed.")
        return results


    def _generate_synthetic_rul(self, labels, **kwargs): # Removed time_proxy
        """Generates synthetic RUL based on labels (0-3)."""
        # --- Function code filled in ---
        print("Generating synthetic RUL (higher category -> lower RUL)...")
        max_rul=kwargs.get('max_rul', 1000)
        damage_impact_factor=kwargs.get('damage_impact_factor', 100)
        rul = np.zeros_like(labels, dtype=float)
        for i, label in enumerate(labels):
            if label == 0: rul[i] = max_rul
            else: rul[i] = max(0, max_rul - damage_impact_factor * label)
        return rul

    def build_lifetime_predictor(self, max_rul=1000, damage_impact=100):
        """Build RUL regression models (Stage 3)."""
        # --- Function code filled in ---
        print("\n--- Building Lifetime Prediction Models (Stage 3 - RUL Regression) ---")
        print(f"NOTE: Using synthetic RUL (max={max_rul}, damage_impact={damage_impact}) for demonstration.")
        for freq in self.frequencies:
            print(f"\nTraining RUL predictors for {freq}...")
            # Check data
            if 'train_features_scaled' not in self.data[freq] or self.data[freq]['train_features_scaled'].empty:
                 print(f" -> Skipping RUL training for {freq}: Features missing/empty."); continue
            X_train_feat = self.data[freq]['train_features_scaled'].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
            y_train_labels = self.data[freq]['train_features_scaled']['damage_category'].values
            X_val_feat = pd.DataFrame(); y_val_rul = np.array([])
            if 'val_features_scaled' in self.data[freq] and not self.data[freq]['val_features_scaled'].empty:
                 X_val_feat = self.data[freq]['val_features_scaled'].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
                 y_val_labels = self.data[freq]['val_features_scaled']['damage_category'].values
                 y_val_rul = self._generate_synthetic_rul(y_val_labels, max_rul=max_rul, damage_impact_factor=damage_impact)
            else: print(f" -> Warning: Validation data missing/empty for {freq}. RUL training without validation.")
            if X_train_feat.empty: print(" -> Skipping: Training features empty."); continue
            y_train_rul = self._generate_synthetic_rul(y_train_labels, max_rul=max_rul, damage_impact_factor=damage_impact)

            # Train RF Regressor
            print("Training Random Forest Regressor..."); rf_regressor = RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_split=5, min_samples_leaf=3, random_state=42, n_jobs=-1)
            rf_regressor.fit(X_train_feat, y_train_rul); self.models['lifetime_predictor'][f'{freq}_random_forest'] = rf_regressor
            if not X_val_feat.empty: val_preds_rf = rf_regressor.predict(X_val_feat); val_mae_rf = mean_absolute_error(y_val_rul, val_preds_rf); print(f" -> RF Regressor Validation MAE: {val_mae_rf:.2f}")
            else: print(" -> RF Validation skipped.")

            # Train XGB Regressor
            print("Training XGBoost Regressor..."); xgb_regressor = xgb.XGBRegressor(n_estimators=150, learning_rate=0.05, max_depth=6, subsample=0.8, colsample_bytree=0.8, objective='reg:squarederror', random_state=42, n_jobs=-1)
            val_set_xgb_rul = [(X_val_feat, y_val_rul)] if not X_val_feat.empty else None
            xgb_regressor.fit(X_train_feat, y_train_rul, eval_set=val_set_xgb_rul, early_stopping_rounds=10 if val_set_xgb_rul else None, verbose=False)
            self.models['lifetime_predictor'][f'{freq}_xgboost'] = xgb_regressor
            if val_set_xgb_rul: val_preds_xgb = xgb_regressor.predict(X_val_feat); val_mae_xgb = mean_absolute_error(y_val_rul, val_preds_xgb); print(f" -> XGB Regressor Validation MAE: {val_mae_xgb:.2f}")
            else: print(" -> XGB Validation skipped.")

            # Train LSTM Regressor
            if 'train_windows_X_scaled' in self.data[freq] and self.data[freq]['train_windows_X_scaled'].size > 0:
                 print("Training LSTM Regressor..."); X_train_win = self.data[freq]['train_windows_X_scaled']; y_train_win_labels = self.data[freq]['train_windows_y']
                 timesteps, n_features = X_train_win.shape[1:]
                 if n_features > 0:
                      y_train_win_rul = self._generate_synthetic_rul(y_train_win_labels, max_rul=max_rul, damage_impact_factor=damage_impact)
                      lstm_regressor = Sequential([ LSTM(128, activation='relu', input_shape=(timesteps, n_features), return_sequences=True), Dropout(0.3), LSTM(64, activation='relu', return_sequences=False), Dropout(0.3), Dense(32, activation='relu'), Dense(1, activation='linear') ])
                      lstm_regressor.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
                      # Prep validation data
                      val_data_lstm_rul = None; callbacks_lstm_rul = None
                      if 'val_windows_X_scaled' in self.data[freq] and self.data[freq]['val_windows_X_scaled'].size > 0:
                           X_val_win = self.data[freq]['val_windows_X_scaled']; y_val_win_labels = self.data[freq]['val_windows_y']
                           y_val_win_rul = self._generate_synthetic_rul(y_val_win_labels, max_rul=max_rul, damage_impact_factor=damage_impact)
                           val_data_lstm_rul = (X_val_win, y_val_win_rul)
                           callbacks_lstm_rul = [EarlyStopping(monitor='val_mae', patience=20, restore_best_weights=True, mode='min')]
                      lstm_regressor.fit(X_train_win, y_train_win_rul, epochs=100, batch_size=64, validation_data=val_data_lstm_rul, callbacks=callbacks_lstm_rul, verbose=0)
                      self.models['lifetime_predictor'][f'{freq}_lstm'] = lstm_regressor
                      if val_data_lstm_rul: val_loss_lstm, val_mae_lstm = lstm_regressor.evaluate(X_val_win, y_val_win_rul, verbose=0); print(f" -> LSTM Regressor Validation MAE: {val_mae_lstm:.2f}")
                      else: print(" -> LSTM Validation skipped.")
                 else: print(" -> Skipping LSTM Regressor: No features in windowed data.")
            else: print("Skipping LSTM Regressor due to missing/empty window data.")
        print("\nLifetime prediction model building completed (RUL Regression).")


    def evaluate_lifetime_predictor(self, max_rul=1000, damage_impact=100):
        """Evaluate RUL regression models (Stage 3)."""
        # --- Function code filled in ---
        print("\n--- Evaluating Lifetime Prediction Models (RUL Regression) ---")
        results = {}
        for freq in self.frequencies:
            print(f"\nEvaluating RUL predictors for {freq}..."); results[freq] = {}
            # Check data
            if 'test_features_scaled' not in self.data[freq] or self.data[freq]['test_features_scaled'].empty:
                 print(f" -> Skipping RUL evaluation for {freq}: Test features missing/empty."); continue
            X_test_feat = self.data[freq]['test_features_scaled'].drop(['damage_category','acquisition_number','damage_category'], axis=1, errors='ignore')
            y_test_labels_feat = self.data[freq]['test_features_scaled']['damage_category'].values
            y_test_rul_feat = self._generate_synthetic_rul(y_test_labels_feat, max_rul=max_rul, damage_impact_factor=damage_impact)
            if X_test_feat.empty: print(" -> Skipping eval: Test features empty."); continue

            # Evaluate RF Regressor
            if f'{freq}_random_forest' in self.models['lifetime_predictor']:
                print("Evaluating Random Forest Regressor..."); rf_regressor = self.models['lifetime_predictor'][f'{freq}_random_forest']
                rf_preds = rf_regressor.predict(X_test_feat)
                results[freq]['Random Forest'] = {'MAE': mean_absolute_error(y_test_rul_feat, rf_preds), 'MSE': mean_squared_error(y_test_rul_feat, rf_preds), 'R2': r2_score(y_test_rul_feat, rf_preds), 'Predictions': rf_preds.tolist()}
                print(f" -> RF Regressor Test MAE: {results[freq]['Random Forest']['MAE']:.2f}, R2: {results[freq]['Random Forest']['R2']:.4f}")
            # Evaluate XGB Regressor
            if f'{freq}_xgboost' in self.models['lifetime_predictor']:
                print("Evaluating XGBoost Regressor..."); xgb_regressor = self.models['lifetime_predictor'][f'{freq}_xgboost']
                xgb_preds = xgb_regressor.predict(X_test_feat)
                results[freq]['XGBoost'] = {'MAE': mean_absolute_error(y_test_rul_feat, xgb_preds), 'MSE': mean_squared_error(y_test_rul_feat, xgb_preds), 'R2': r2_score(y_test_rul_feat, xgb_preds), 'Predictions': xgb_preds.tolist()}
                print(f" -> XGB Regressor Test MAE: {results[freq]['XGBoost']['MAE']:.2f}, R2: {results[freq]['XGBoost']['R2']:.4f}")
            # Evaluate LSTM Regressor
            if f'{freq}_lstm' in self.models['lifetime_predictor']:
                 print("Evaluating LSTM Regressor...")
                 lstm_regressor = self.models['lifetime_predictor'][f'{freq}_lstm']
                 if 'test_windows_X_scaled' in self.data[freq] and self.data[freq]['test_windows_X_scaled'].size > 0:
                     X_test_win = self.data[freq]['test_windows_X_scaled']; y_test_labels_win = self.data[freq]['test_windows_y']
                     y_test_rul_win = self._generate_synthetic_rul(y_test_labels_win, max_rul=max_rul, damage_impact_factor=damage_impact)
                     lstm_preds = lstm_regressor.predict(X_test_win, verbose=0).flatten()
                     results[freq]['LSTM'] = {'MAE': mean_absolute_error(y_test_rul_win, lstm_preds), 'MSE': mean_squared_error(y_test_rul_win, lstm_preds), 'R2': r2_score(y_test_rul_win, lstm_preds), 'Predictions': lstm_preds.tolist()}
                     print(f" -> LSTM Regressor Test MAE: {results[freq]['LSTM']['MAE']:.2f}, R2: {results[freq]['LSTM']['R2']:.4f}")
                 else: print(" -> Skipping LSTM RUL eval: Test window data missing/empty.")

        # Store RUL results (excluding CoxPH results which are handled separately)
        rul_eval_results = {freq: {k: v for k, v in res.items() if k != 'CoxPH'} for freq, res in results.items()}
        if 'RUL' not in self.evaluation_results['lifetime_prediction']:
            self.evaluation_results['lifetime_prediction']['RUL'] = {}
        self.evaluation_results['lifetime_prediction']['RUL'].update(rul_eval_results)
        print("\nLifetime prediction evaluation completed (RUL Regression).")
        return rul_eval_results # Return only RUL results from this function


    # --- Survival Analysis methods unchanged ---
    def build_survival_model(self):
        # --- Function code filled in (same as before) ---
        print("\n--- Building Survival Analysis Model (Stage 3 - CoxPH) ---")
        print("CAUTION: Using simplified survival data (window acquisition as time, category>0 as event).")
        for freq in self.frequencies:
            print(f"\nTraining CoxPH model for {freq}...")
            df_key = 'train_features_scaled'
            if df_key not in self.data[freq] or self.data[freq][df_key].empty: print(f" -> Skipping CoxPH training for {freq}: Features missing/empty."); continue
            survival_df = self.data[freq][df_key].copy()
            survival_df['duration'] = survival_df['acquisition_number']
            survival_df['event'] = (survival_df['damage_category'] > 0).astype(int)
            covariate_cols = [col for col in survival_df.columns if col not in ['acquisition_number', 'damage_category', 'duration', 'event', 'damage_category']]
            if not covariate_cols: print(f" -> Skipping CoxPH training for {freq}: No covariates."); continue
            if survival_df['duration'].nunique() < 2 or survival_df['event'].nunique() < 2: print(f" -> Skipping CoxPH training for {freq}: Not enough variability."); continue
            survival_df = survival_df[['duration', 'event'] + covariate_cols].replace([np.inf, -np.inf], np.nan).dropna(axis=0)
            if survival_df.empty: print(f" -> Skipping CoxPH training for {freq}: Data empty after dropna."); continue
            cph = CoxPHFitter(penalizer=0.1)
            try: cph.fit(survival_df, duration_col='duration', event_col='event'); self.models['lifetime_predictor'][f'{freq}_coxph'] = cph; print(f" -> CoxPH model trained for {freq}."); cph.print_summary(decimals=3)
            except Exception as e: print(f"Error training CoxPH model for {freq}: {e}")
        print("\nSurvival Analysis model building completed.")

    def evaluate_survival_model(self):
        # --- Function code filled in (same as before) ---
        print("\n--- Evaluating Survival Analysis Model (CoxPH) ---")
        results_survival = {}
        for freq in self.frequencies:
             print(f"\nEvaluating CoxPH model for {freq}..."); model_key = f'{freq}_coxph'; df_key = 'test_features_scaled'
             if model_key not in self.models['lifetime_predictor']: print(f" -> Skipping CoxPH eval for {freq}: Model not found."); continue
             if df_key not in self.data[freq] or self.data[freq][df_key].empty: print(f" -> Skipping CoxPH eval for {freq}: Test features missing/empty."); continue
             cph = self.models['lifetime_predictor'][model_key]; test_survival_df = self.data[freq][df_key].copy()
             test_survival_df['duration'] = test_survival_df['acquisition_number']; test_survival_df['event'] = (test_survival_df['damage_category'] > 0).astype(int)
             covariate_cols = [col for col in test_survival_df.columns if col not in ['acquisition_number', 'damage_category', 'duration', 'event', 'damage_category']]
             if not covariate_cols: print(f" -> Skipping CoxPH eval for {freq}: No covariates."); continue
             test_survival_df = test_survival_df[['duration', 'event'] + covariate_cols].replace([np.inf, -np.inf], np.nan).dropna(axis=0)
             if test_survival_df.empty: print(f" -> Skipping CoxPH eval for {freq}: Test data empty after dropna."); continue
             try: c_index = cph.score(test_survival_df, scoring_method="concordance_index"); results_survival[freq] = {'Concordance Index': c_index}; print(f" -> CoxPH Concordance Index for {freq}: {c_index:.4f}")
             except Exception as e: print(f"Error evaluating CoxPH model for {freq}: {e}"); results_survival[freq] = {'Concordance Index': np.nan}
        if 'CoxPH' not in self.evaluation_results['lifetime_prediction']: self.evaluation_results['lifetime_prediction']['CoxPH'] = {}
        self.evaluation_results['lifetime_prediction']['CoxPH'].update(results_survival)
        print("\nSurvival Analysis model evaluation completed.")
        return results_survival


    # --- Visualization Methods (Filled in) ---
    def visualize_data(self):
        """Generates standard visualizations (feature/split distributions)."""
        # --- Function code filled in (same as before) ---
        print("\n--- Creating Visualizations ---")
        os.makedirs(VIZ_SAVE_DIR, exist_ok=True)
        for freq in self.frequencies:
            print(f"\nGenerating standard plots for {freq}...")
            # Plot feature distributions
            df_key = 'train_features_df'; label_col = 'damage_category'
            if df_key not in self.data[freq] or self.data[freq][df_key].empty: print(f" -> Skipping feature dist plot for {freq}: Data missing/empty.")
            else:
                 feature_df = self.data[freq][df_key]; plt.figure(figsize=(18, 12)); plt.suptitle(f'Feature Distributions - {freq} (Train Set, Unscaled)', fontsize=16)
                 features_to_plot = [col for col in feature_df.columns if col not in ['acquisition_number', label_col]][:9]
                 num_plots = len(features_to_plot)
                 if num_plots > 0:
                      num_cols = 3; num_rows = (num_plots + num_cols - 1) // num_cols; colors = plt.cm.viridis(np.linspace(0, 1, len(self.crack_categories)))
                      for i, feature in enumerate(features_to_plot):
                           ax = plt.subplot(num_rows, num_cols, i + 1)
                           for cat_code, cat_name in self.crack_categories.items():
                                subset = feature_df[feature_df[label_col] == cat_code][feature]
                                if not subset.empty: sns.histplot(subset, kde=True, label=cat_name, ax=ax, color=colors[cat_code], stat="density", common_norm=False, element="step")
                           ax.set_title(f'{feature}'); ax.set_xlabel(''); ax.set_ylabel('Density'); ax.legend()
                      plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_feature_distributions_4cat.png')); print(f" -> Saved feature dist plot for {freq}.")
                 else: print(f" -> No features to plot distributions for {freq}.")
                 plt.close()
            # Plot class distribution in splits
            all_counts = {}; splits_exist = False
            for split_name in ['train', 'val', 'test']:
                 df_key = f'{split_name}_features_df'
                 if df_key in self.data[freq] and not self.data[freq][df_key].empty and 'damage_category' in self.data[freq][df_key].columns:
                      all_counts[split_name.capitalize()] = self.data[freq][df_key]['damage_category'].value_counts(); splits_exist = True
                 else: all_counts[split_name.capitalize()] = pd.Series(dtype=int)
            if not splits_exist: print(f" -> Skipping split dist plot for {freq}: No split data.")
            else:
                 count_df = pd.DataFrame(all_counts).fillna(0).astype(int); count_df = count_df.reindex(list(self.crack_categories.keys()), fill_value=0)
                 count_df.index = count_df.index.map(self.crack_categories)
                 count_df.plot(kind='bar', figsize=(12, 7), width=0.8); plt.ylabel('Count'); plt.xlabel('Damage Category'); plt.title(f'Class Distribution in Splits - {freq}')
                 plt.xticks(rotation=0); plt.legend(title='Dataset Split'); plt.tight_layout(); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_class_distribution_split_4cat.png')); print(f" -> Saved class dist plot for {freq}.")
                 plt.close()
        print("Standard visualization generation finished.")

    def visualize_feature_importance(self):
        """Plots feature importances from the Random Forest classifier."""
        # --- Function code filled in (same as before) ---
        print("\n--- Generating Feature Importance Plots ---")
        for freq in self.frequencies:
            imp_key = f'{freq}_crack_classification_rf'
            if imp_key in self.feature_importances:
                print(f"Plotting feature importance for {freq}...")
                importances = self.feature_importances[imp_key]
                if isinstance(importances, pd.Series) and not importances.empty:
                    plt.figure(figsize=(10, 8)); importances.nlargest(20).plot(kind='barh'); plt.title(f'Top 20 Feature Importances (RF Classifier) - {freq}'); plt.xlabel('Importance'); plt.gca().invert_yaxis(); plt.tight_layout(); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_feature_importance.png')); plt.close()
                    print(f" -> Saved feature importance plot for {freq}.")
                else: print(f" -> Skipping importance plot for {freq}: Data invalid.")
            else: print(f" -> Skipping importance plot for {freq}: Data not found.")

    def visualize_anomalies(self):
        """Visualizes anomalies detected on the test set."""
        # --- Function code filled in (same as before) ---
        print("\n--- Generating Anomaly Detection Plots ---")
        for freq in self.frequencies:
            print(f"Plotting anomalies for {freq}...")
            if freq not in self.evaluation_results.get('anomaly_detection', {}) or 'test_features_df' not in self.data[freq] or self.data[freq]['test_features_df'].empty: print(f" -> Skipping anomaly plot for {freq}: Missing results/data."); continue
            results_ad = self.evaluation_results['anomaly_detection'][freq]; test_df_orig = self.data[freq]['test_features_df']
            # Plot IF predictions
            model_name_if = 'Isolation Forest'
            if model_name_if in results_ad and 'predictions' in results_ad[model_name_if]:
                if len(results_ad[model_name_if]['predictions']) == len(test_df_orig):
                    test_df = test_df_orig.copy(); test_df['anomaly_pred_if'] = results_ad[model_name_if]['predictions']; plt.figure(figsize=(15, 6))
                    feature_to_plot = 'torsional_peak_to_peak'; # Example feature
                    if feature_to_plot not in test_df.columns: feature_to_plot = test_df.drop(['acquisition_number', 'damage_category', 'anomaly_pred_if'], axis=1).columns[0]
                    sns.scatterplot(data=test_df, x='acquisition_number', y=feature_to_plot, hue='anomaly_pred_if', style='damage_category', palette={0: 'blue', 1: 'red'}); plt.title(f'Anomaly Detection ({model_name_if}) vs {feature_to_plot} - {freq}'); plt.xlabel('Acquisition Number'); plt.ylabel(f'{feature_to_plot} (Unscaled)'); plt.legend(title=f'Anomaly (IF) | True Cat'); plt.tight_layout(); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_anomalies_{model_name_if}.png')); plt.close(); print(f" -> Saved anomaly plot ({model_name_if}) for {freq}.")
                else: print(f" -> Skipping IF anomaly plot for {freq}: Length mismatch.")
            else: print(f" -> Skipping IF anomaly plot for {freq}: Predictions not found.")
            # Plot AE Error
            model_name_ae = 'Dense Autoencoder'
            if model_name_ae in results_ad and 'reconstruction_error' in results_ad[model_name_ae]:
                 if len(results_ad[model_name_ae]['reconstruction_error']) == len(test_df_orig):
                      test_df = test_df_orig.copy(); test_df['ae_error'] = results_ad[model_name_ae]['reconstruction_error']; threshold = results_ad[model_name_ae].get('Threshold', None)
                      plt.figure(figsize=(15, 6)); sns.scatterplot(data=test_df, x='acquisition_number', y='ae_error', hue='damage_category', palette='viridis')
                      if threshold is not None: plt.axhline(threshold, color='red', linestyle='--', label=f'Threshold ({threshold:.4f})')
                      plt.title(f'AE Reconstruction Error - {freq}'); plt.xlabel('Acquisition Number'); plt.ylabel('MSE'); plt.legend(); plt.tight_layout(); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_anomaly_ae_error.png')); plt.close(); print(f" -> Saved AE error plot for {freq}.")
                 else: print(f" -> Skipping AE error plot for {freq}: Length mismatch.")
            else: print(f" -> Skipping AE error plot for {freq}: Errors not found.")


    def visualize_classification_results(self):
        """Plots confusion matrices for the 4-category classification."""
        # --- Function code filled in (same as before) ---
        print("\n--- Generating Classification Result Plots ---")
        for freq in self.frequencies:
            print(f"Plotting classification results for {freq}...")
            if freq not in self.evaluation_results.get('crack_classification', {}): print(f" -> Skipping classification plots for {freq}: Results not found."); continue
            results_cls = self.evaluation_results['crack_classification'][freq]; labels_present = list(range(len(self.crack_categories))); target_names_present = [self.crack_categories.get(i, str(i)) for i in labels_present]
            # Try to get actual labels present from one of the reports if available
            if 'Random Forest' in results_cls and isinstance(results_cls['Random Forest'].get('Confusion Matrix Labels'), list): labels_present = results_cls['Random Forest']['Confusion Matrix Labels']; target_names_present = [self.crack_categories.get(i, str(i)) for i in labels_present]
            for model_name, results in results_cls.items():
                if 'Confusion Matrix' in results and isinstance(results['Confusion Matrix'], list):
                    cm = np.array(results['Confusion Matrix']); # Check if list is nested correctly
                    if cm.ndim == 2 and cm.size > 0:
                         plt.figure(figsize=(8, 6)); sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names_present, yticklabels=target_names_present)
                         plt.title(f'Confusion Matrix - {model_name} - {freq}'); plt.xlabel('Predicted Label'); plt.ylabel('True Label'); plt.tight_layout(); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_classification_CM_{model_name}.png')); plt.close(); print(f" -> Saved CM plot ({model_name}) for {freq}.")
                    else: print(f" -> Skipping CM plot for {freq} - {model_name}: Matrix invalid.")
                else: print(f" -> Skipping CM plot for {freq} - {model_name}: Matrix not found.")


    def visualize_rul_predictions(self):
        """Plots actual vs predicted RUL over acquisition number for the test set."""
        # --- Function code filled in (same as before) ---
        print("\n--- Generating RUL Prediction Plots ---")
        for freq in self.frequencies:
            print(f"Plotting RUL predictions for {freq}...")
            if freq not in self.evaluation_results.get('lifetime_prediction', {}).get('RUL', {}) or 'test_features_df' not in self.data[freq] or self.data[freq]['test_features_df'].empty: print(f" -> Skipping RUL plot for {freq}: Missing results/data."); continue
            results_rul = self.evaluation_results['lifetime_prediction']['RUL'][freq]; test_df = self.data[freq]['test_features_df'].copy()
            test_labels = test_df['damage_category'].values; true_rul = self._generate_synthetic_rul(test_labels); test_df['true_rul'] = true_rul
            plt.figure(figsize=(15, 7)); plt.plot(test_df['acquisition_number'], test_df['true_rul'], label='True Synthetic RUL', color='black', linestyle='--', linewidth=2, zorder=1)
            plot_exists = False
            for model_name, results in results_rul.items():
                 if isinstance(results, dict) and 'Predictions' in results:
                      predictions = results['Predictions']
                      if len(predictions) == len(test_df): plt.scatter(test_df['acquisition_number'], predictions, label=f'Predicted RUL ({model_name})', alpha=0.6, s=10, zorder=2); plot_exists = True
                      else: print(f" -> Warning: RUL prediction length mismatch for {model_name} - {freq}.")
            if not plot_exists: print(f" -> No RUL predictions found to plot for {freq}."); plt.close(); continue
            plt.title(f'True vs Predicted Synthetic RUL - {freq}'); plt.xlabel('Acquisition Number'); plt.ylabel('Remaining Useful Life (Synthetic)'); plt.legend(); plt.grid(True, linestyle=':', alpha=0.6); plt.tight_layout(); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_RUL_predictions.png')); plt.close(); print(f" -> Saved RUL prediction plot for {freq}.")

    def visualize_survival_analysis(self):
        """Creates visualizations for the Cox Proportional Hazards model."""
        # --- Function code filled in (same as before) ---
        print("\n--- Creating Survival Analysis Visualizations ---")
        os.makedirs(VIZ_SAVE_DIR, exist_ok=True)
        for freq in self.frequencies:
             model_key = f'{freq}_coxph'
             if model_key not in self.models['lifetime_predictor']: print(f" -> Skipping survival viz for {freq}: Model not found."); continue
             print(f"Generating survival plots for {freq}..."); cph = self.models['lifetime_predictor'][model_key]
             # Plot Coefficients
             try: plt.figure(figsize=(10, 6)); cph.plot(); plt.title(f'CoxPH Coefficients - {freq}'); plt.tight_layout(); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_coxph_coefficients.png')); plt.close(); print(f" -> Saved CoxPH coefficients plot for {freq}.")
             except Exception as e: print(f"Error plotting CoxPH coeffs for {freq}: {e}"); plt.close()
             # Plot Partial Effects
             try:
                  summary_df = cph.summary; significant_features = summary_df[summary_df['p'] < 0.05].sort_values(by='coef', key=abs, ascending=False).index.tolist()
                  if significant_features:
                      num_partial_plots = min(len(significant_features), 3); print(f"Plotting partial effects for: {significant_features[:num_partial_plots]}")
                      plt.figure(figsize=(7 * num_partial_plots, 6)); cph.plot_partial_effects_on_outcome(covariates=significant_features[:num_partial_plots], values=np.percentile(cph.data[significant_features[:num_partial_plots]], [10, 50, 90], axis=0).T, plot_baseline=True)
                      plt.suptitle(f'Partial Effects on Outcome - {freq}'); plt.tight_layout(rect=[0, 0.03, 1, 0.95]); plt.savefig(os.path.join(VIZ_SAVE_DIR, f'{freq}_coxph_partial_effects.png')); plt.close(); print(f" -> Saved CoxPH partial effects plot for {freq}.")
                  else: print(f" -> No significant features (p<0.05) for partial effects plot for {freq}.")
             except Exception as e: print(f"Error plotting CoxPH partial effects for {freq}: {e}"); plt.close()
        print("Survival analysis visualization generation finished.")


    # --- Standard Methods: save_models, load_models ---
    def save_models(self, model_dir=MODEL_SAVE_DIR, scaler_dir=SCALER_SAVE_DIR):
        """Saves trained models and scalers."""
        # --- Function code filled in (same as before) ---
        print("\n--- Saving Models and Scalers ---")
        os.makedirs(model_dir, exist_ok=True); os.makedirs(scaler_dir, exist_ok=True)
        for name, scaler in self.scalers.items():
             if scaler: joblib.dump(scaler, f"{scaler_dir}/{name}_scaler.joblib"); print(f"Scaler '{name}' saved.")
        for stage, stage_models in self.models.items():
            for name, model in stage_models.items():
                save_path = f"{model_dir}/{name}.model"
                try:
                    if isinstance(model, (RandomForestClassifier, RandomForestRegressor, IsolationForest, OneClassSVM, xgb.XGBClassifier, xgb.XGBRegressor, CoxPHFitter)): # Added CoxPHFitter
                        joblib.dump(model, save_path + '.joblib'); print(f"Model '{name}' saved as joblib.")
                    elif isinstance(model, (Model, Sequential)):
                        model.save(save_path + '.keras'); print(f"Model '{name}' saved as Keras format.")
                    else: print(f"Warning: Model type for '{name}' not recognized for saving.")
                except Exception as e: print(f"Error saving model {name}: {e}")
        print("Model and scaler saving process finished.")


    def load_models(self, model_dir=MODEL_SAVE_DIR, scaler_dir=SCALER_SAVE_DIR):
        """Loads previously saved models and scalers."""
        # --- Function code filled in (same as before) ---
        print("\n--- Loading Models and Scalers ---")
        try: # Load Scalers
            if os.path.isdir(scaler_dir):
                for f in os.listdir(scaler_dir):
                     if f.endswith('_scaler.joblib'): name = f.replace('_scaler.joblib', ''); self.scalers[name] = joblib.load(f"{scaler_dir}/{f}"); print(f"Scaler '{name}' loaded.")
            else: print(f"Scaler directory not found: {scaler_dir}")
        except Exception as e: print(f"Error loading scalers: {e}")
        try: # Load Models
            if os.path.isdir(model_dir):
                 for f in os.listdir(model_dir):
                      load_path = os.path.join(model_dir, f); model_name = f.replace('.model.joblib', '').replace('.model.keras', '')
                      stage = None # Determine stage
                      if 'isolation_forest' in model_name or 'ocsvm' in model_name or 'autoencoder' in model_name: stage = 'anomaly_detector'
                      elif 'random_forest' in model_name or 'xgboost' in model_name or 'neural_network' in model_name or 'coxph' in model_name: stage = 'lifetime_predictor' if 'lifetime_predictor' in self.models and (f'{model_name}' in self.models['lifetime_predictor'] or 'coxph' in model_name) else 'crack_classifier'
                      elif 'lstm' in model_name: stage = 'lifetime_predictor' if 'lifetime_predictor' in self.models and f'{model_name}' in self.models['lifetime_predictor'] else 'anomaly_detector'
                      if stage:
                           try:
                                if f.endswith('.joblib'):
                                     loaded_model = joblib.load(load_path); self.models[stage][model_name] = loaded_model
                                     model_type = "CoxPH" if isinstance(loaded_model, CoxPHFitter) else "sklearn/xgb/joblib"
                                     print(f"Model '{model_name}' ({model_type}) loaded for stage '{stage}'.")
                                elif f.endswith('.keras'): self.models[stage][model_name] = load_model(load_path); print(f"Model '{model_name}' (Keras) loaded for stage '{stage}'.")
                                else: print(f"Skipping format: {f}")
                           except Exception as e: print(f"Error loading model {f}: {e}")
            else: print(f"Model directory not found: {model_dir}")
        except Exception as e: print(f"Error loading models: {e}")
        print("Model and scaler loading process finished.")


    def run_pipeline(self, perform_training=True, perform_evaluation=True, run_survival_analysis=True,
                     save_models_on_completion=True, generate_visualizations=True):
        """
        Run the full analysis pipeline with integrated visualizations.
        """
        # --- Function code filled in (same as before) ---
        try:
             self.load_data()
             self.preprocess_features()
             if perform_training:
                 self.build_anomaly_detector()
                 self.build_crack_classifier()
                 self.build_lifetime_predictor() # RUL Regressors
                 if run_survival_analysis: self.build_survival_model() # CoxPH
                 if save_models_on_completion: self.save_models()
             else: print("\n--- Skipping Training - Loading Models ---"); self.load_models()
             if perform_evaluation:
                 print("\n--- Running Evaluation ---")
                 self.evaluate_anomaly_detector()
                 self.evaluate_crack_classifier()
                 self.evaluate_lifetime_predictor() # RUL Regressors
                 if run_survival_analysis and any('coxph' in k for k in self.models.get('lifetime_predictor',{})): self.evaluate_survival_model() # CoxPH
                 print("\n--- Evaluation Summary ---") # Optional summary
             if generate_visualizations:
                 print("\n--- Generating All Visualizations ---")
                 self.visualize_data() # Feature/Split Dist
                 self.visualize_feature_importance() # RF Importance
                 self.visualize_anomalies() # Anomaly Points/Error
                 self.visualize_classification_results() # Confusion Matrices
                 self.visualize_rul_predictions() # RUL Plot
                 if run_survival_analysis and any('coxph' in k for k in self.models.get('lifetime_predictor', {})): self.visualize_survival_analysis() # Cox Plots
             print("\n--- Pipeline Run Finished ---")
        except FileNotFoundError as e: print(f"Pipeline stopped - Missing file: {e}")
        except Exception as e: print(f"Pipeline stopped - Error: {e}"); raise


# --- Example Usage ---
if __name__ == "__main__":
    frequencies_to_run = ['Frequency_1', 'Frequency_2','Frequency_3', 'Frequency_4','Frequency_5'] # Adjust as needed
    pipeline = PipelineCrackAnalysisPipeline( data_dir='./processed_data/', frequencies=frequencies_to_run )
    pipeline.run_pipeline( perform_training=True, perform_evaluation=True, run_survival_analysis=True, save_models_on_completion=True, generate_visualizations=True )

--- Loading Data ---
Loading data for Frequency_1...
 -> Loaded windowed NPY data.
 -> Loaded feature CSV data.
 -> Windowed data shapes (Train X/y, Val X/y, Test X/y):
    (196, 5, 17), (196,)
Loading data for Frequency_2...
 -> Loaded windowed NPY data.
 -> Loaded feature CSV data.
 -> Windowed data shapes (Train X/y, Val X/y, Test X/y):
    (196, 5, 17), (196,)
Loading data for Frequency_3...
 -> Loaded windowed NPY data.
 -> Loaded feature CSV data.
 -> Windowed data shapes (Train X/y, Val X/y, Test X/y):
    (196, 5, 17), (196,)
Loading data for Frequency_4...
 -> Loaded windowed NPY data.
 -> Loaded feature CSV data.
 -> Windowed data shapes (Train X/y, Val X/y, Test X/y):
    (196, 5, 17), (196,)
Loading data for Frequency_5...
 -> Loaded windowed NPY data.
 -> Loaded feature CSV data.
 -> Windowed data shapes (Train X/y, Val X/y, Test X/y):
    (196, 5, 17), (196,)
Data loaded successfully.

--- Feature Preprocessing ---
Creating scaled feature set for Frequency_1...
 -> Fitted

<lifelines.CoxPHFitter: fitted with 200 total observations, 181 right-censored observations>
             duration col = 'duration'
                event col = 'event'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 200
number of events observed = 19
   partial log-likelihood = -35.646
         time fit was run = 2025-04-02 23:03:48 UTC

---
                         coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                
torsional_mean          0.013     1.013     0.221          -0.420           0.447               0.657               1.564
torsional_std           0.114     1.121     0.210          -0.297           0.525               0.743               1.691
torsional_max           0.085     1.089     0.209          -0.324           0.495               0.723               1.640
torsional_min          -0.067     0.935     0.209          -0.477           0.343               0.621               1.410
torsional_peak_to_peak  0.076     1.079     0.209          -0.334           0.487               0.716               1.627
torsional_rms           0.114     1.121     0.210          -0.297           0.525               0.743               1.691
torsional_kurtosis     -0.153     0.858     0.196          -0.536           0.230               0.585               1.259
flexural_mean           0.058     1.060     0.213          -0.360           0.475               0.698               1.609
flexural_std            0.092     1.096     0.212          -0.324           0.507               0.723               1.661
flexural_max            0.038     1.038     0.210          -0.374           0.450               0.688               1.568
flexural_min           -0.029     0.971     0.209          -0.438           0.380               0.645               1.462
flexural_peak_to_peak   0.034     1.034     0.210          -0.377           0.444               0.686               1.559
flexural_rms            0.092     1.096     0.212          -0.324           0.507               0.723               1.661
flexural_kurtosis      -0.135     0.873     0.184          -0.496           0.225               0.609               1.252
torsional_energy        0.149     1.161     0.212          -0.266           0.564               0.767               1.758
flexural_energy         0.076     1.079     0.215          -0.345           0.498               0.708               1.645
energy_ratio           -0.001     0.999     0.200          -0.392           0.391               0.676               1.478

                        cmp to      z     p  -log2(p)
covariate                                            
torsional_mean           0.000  0.060 0.952     0.071
torsional_std            0.000  0.545 0.586     0.771
torsional_max            0.000  0.408 0.683     0.550
torsional_min            0.000 -0.320 0.749     0.416
torsional_peak_to_peak   0.000  0.366 0.715     0.485
torsional_rms            0.000  0.545 0.586     0.771
torsional_kurtosis       0.000 -0.782 0.434     1.203
flexural_mean            0.000  0.271 0.786     0.347
flexural_std             0.000  0.433 0.665     0.588
flexural_max             0.000  0.179 0.858     0.222
flexural_min             0.000 -0.140 0.889     0.170
flexural_peak_to_peak    0.000  0.160 0.873     0.196
flexural_rms             0.000  0.433 0.665     0.588
flexural_kurtosis        0.000 -0.737 0.461     1.116
torsional_energy         0.000  0.704 0.481     1.055
flexural_energy          0.000  0.356 0.722     0.470
energy_ratio             0.000 -0.004 0.997     0.004
---
Concordance = 0.784
Partial AIC = 105.291
log-likelihood ratio test = 7.388 on 17 df
-log2(p) of ll-ratio test = 0.032


Training CoxPH model for Frequency_2...
 -> CoxPH model trained for Frequency_2.


<lifelines.CoxPHFitter: fitted with 200 total observations, 181 right-censored observations>
             duration col = 'duration'
                event col = 'event'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 200
number of events observed = 19
   partial log-likelihood = -32.095
         time fit was run = 2025-04-02 23:03:48 UTC

---
                         coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                
torsional_mean         -0.004     0.996     0.184          -0.365           0.356               0.694               1.428
torsional_std           0.139     1.149     0.212          -0.277           0.554               0.758               1.741
torsional_max           0.027     1.028     0.205          -0.374           0.429               0.688               1.535
torsional_min          -0.052     0.950     0.206          -0.455           0.352               0.634               1.422
torsional_peak_to_peak  0.040     1.040     0.205          -0.363           0.442               0.696               1.556
torsional_rms           0.139     1.149     0.212          -0.277           0.554               0.758               1.741
torsional_kurtosis     -0.190     0.827     0.196          -0.574           0.194               0.563               1.214
flexural_mean          -0.086     0.917     0.193          -0.465           0.292               0.628               1.339
flexural_std            0.077     1.080     0.215          -0.345           0.499               0.708               1.648
flexural_max           -0.007     0.993     0.217          -0.431           0.418               0.650               1.519
flexural_min            0.002     1.002     0.217          -0.424           0.427               0.655               1.532
flexural_peak_to_peak  -0.004     0.996     0.217          -0.429           0.421               0.651               1.523
flexural_rms            0.077     1.080     0.215          -0.345           0.499               0.708               1.648
flexural_kurtosis      -0.300     0.741     0.184          -0.662           0.061               0.516               1.063
torsional_energy        0.163     1.177     0.213          -0.255           0.581               0.775               1.787
flexural_energy         0.069     1.072     0.217          -0.357           0.495               0.700               1.641
energy_ratio            0.050     1.051     0.200          -0.342           0.442               0.710               1.555

                        cmp to      z     p  -log2(p)
covariate                                            
torsional_mean           0.000 -0.024 0.981     0.027
torsional_std            0.000  0.654 0.513     0.962
torsional_max            0.000  0.133 0.894     0.162
torsional_min            0.000 -0.252 0.801     0.319
torsional_peak_to_peak   0.000  0.193 0.847     0.240
torsional_rms            0.000  0.654 0.513     0.962
torsional_kurtosis       0.000 -0.971 0.331     1.593
flexural_mean            0.000 -0.447 0.655     0.611
flexural_std             0.000  0.358 0.720     0.474
flexural_max             0.000 -0.030 0.976     0.035
flexural_min             0.000  0.007 0.994     0.008
flexural_peak_to_peak    0.000 -0.019 0.985     0.022
flexural_rms             0.000  0.358 0.720     0.474
flexural_kurtosis        0.000 -1.627 0.104     3.269
torsional_energy         0.000  0.764 0.445     1.168
flexural_energy          0.000  0.320 0.749     0.416
energy_ratio             0.000  0.249 0.803     0.316
---
Concordance = 0.825
Partial AIC = 98.189
log-likelihood ratio test = 14.491 on 17 df
-log2(p) of ll-ratio test = 0.662


Training CoxPH model for Frequency_3...
 -> CoxPH model trained for Frequency_3.


...


Training CoxPH model for Frequency_4...
 -> CoxPH model trained for Frequency_4.


<lifelines.CoxPHFitter: fitted with 200 total observations, 181 right-censored observations>
             duration col = 'duration'
                event col = 'event'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 200
number of events observed = 19
   partial log-likelihood = -34.886
         time fit was run = 2025-04-02 23:03:48 UTC

---
                         coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                
torsional_mean          0.088     1.092     0.173          -0.250           0.426               0.779               1.531
torsional_std          -0.036     0.965     0.208          -0.444           0.372               0.641               1.451
torsional_max           0.110     1.116     0.214          -0.310           0.530               0.733               1.699
torsional_min          -0.084     0.920     0.214          -0.503           0.336               0.605               1.399
torsional_peak_to_peak  0.097     1.102     0.214          -0.323           0.517               0.724               1.677
torsional_rms          -0.036     0.965     0.208          -0.444           0.372               0.641               1.451
torsional_kurtosis      0.184     1.202     0.204          -0.215           0.583               0.807               1.792
flexural_mean           0.053     1.055     0.150          -0.241           0.348               0.786               1.417
flexural_std           -0.079     0.924     0.207          -0.485           0.328               0.616               1.388
flexural_max            0.009     1.009     0.213          -0.408           0.425               0.665               1.530
flexural_min           -0.027     0.973     0.213          -0.446           0.391               0.640               1.478
flexural_peak_to_peak   0.018     1.018     0.213          -0.400           0.435               0.671               1.545
flexural_rms           -0.079     0.924     0.207          -0.485           0.328               0.616               1.388
flexural_kurtosis       0.240     1.272     0.186          -0.124           0.605               0.883               1.831
torsional_energy       -0.085     0.919     0.198          -0.473           0.304               0.623               1.355
flexural_energy        -0.046     0.955     0.219          -0.475           0.383               0.622               1.466
energy_ratio            0.142     1.153     0.202          -0.254           0.538               0.776               1.713

                        cmp to      z     p  -log2(p)
covariate                                            
torsional_mean           0.000  0.510 0.610     0.713
torsional_std            0.000 -0.173 0.863     0.213
torsional_max            0.000  0.512 0.609     0.716
torsional_min            0.000 -0.391 0.696     0.523
torsional_peak_to_peak   0.000  0.452 0.651     0.619
torsional_rms            0.000 -0.173 0.863     0.213
torsional_kurtosis       0.000  0.904 0.366     1.450
flexural_mean            0.000  0.356 0.722     0.470
flexural_std             0.000 -0.379 0.705     0.505
flexural_max             0.000  0.042 0.966     0.049
flexural_min             0.000 -0.128 0.898     0.155
flexural_peak_to_peak    0.000  0.084 0.933     0.100
flexural_rms             0.000 -0.379 0.705     0.505
flexural_kurtosis        0.000  1.292 0.196     2.349
torsional_energy         0.000 -0.426 0.670     0.578
flexural_energy          0.000 -0.211 0.833     0.263
energy_ratio             0.000  0.704 0.481     1.055
---
Concordance = 0.766
Partial AIC = 103.771
log-likelihood ratio test = 8.909 on 17 df
-log2(p) of ll-ratio test = 0.085


Training CoxPH model for Frequency_5...
 -> CoxPH model trained for Frequency_5.


<lifelines.CoxPHFitter: fitted with 200 total observations, 181 right-censored observations>
             duration col = 'duration'
                event col = 'event'
                penalizer = 0.1
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 200
number of events observed = 19
   partial log-likelihood = -35.360
         time fit was run = 2025-04-02 23:03:48 UTC

---
                         coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                                
torsional_mean          0.162     1.175     0.211          -0.251           0.574               0.778               1.776
torsional_std          -0.050     0.951     0.213          -0.468           0.367               0.626               1.443
torsional_max           0.020     1.020     0.211          -0.394           0.434               0.674               1.543
torsional_min          -0.041     0.960     0.212          -0.457           0.375               0.633               1.455
torsional_peak_to_peak  0.030     1.031     0.212          -0.385           0.445               0.681               1.561
torsional_rms          -0.050     0.951     0.213          -0.468           0.367               0.626               1.443
torsional_kurtosis      0.168     1.183     0.212          -0.249           0.584               0.780               1.793
flexural_mean          -0.027     0.973     0.185          -0.390           0.336               0.677               1.399
flexural_std           -0.145     0.865     0.212          -0.561           0.272               0.570               1.312
flexural_max           -0.057     0.944     0.208          -0.466           0.351               0.628               1.421
flexural_min            0.068     1.070     0.206          -0.337           0.472               0.714               1.603
flexural_peak_to_peak  -0.062     0.939     0.207          -0.469           0.344               0.626               1.411
flexural_rms           -0.145     0.865     0.212          -0.561           0.272               0.570               1.312
flexural_kurtosis       0.148     1.159     0.195          -0.233           0.529               0.792               1.698
torsional_energy       -0.057     0.944     0.208          -0.465           0.351               0.628               1.420
flexural_energy        -0.103     0.902     0.215          -0.524           0.319               0.592               1.376
energy_ratio            0.113     1.119     0.205          -0.289           0.514               0.749               1.673

                        cmp to      z     p  -log2(p)
covariate                                            
torsional_mean           0.000  0.767 0.443     1.174
torsional_std            0.000 -0.237 0.813     0.299
torsional_max            0.000  0.094 0.925     0.112
torsional_min            0.000 -0.193 0.847     0.240
torsional_peak_to_peak   0.000  0.143 0.886     0.174
torsional_rms            0.000 -0.237 0.813     0.299
torsional_kurtosis       0.000  0.790 0.430     1.219
flexural_mean            0.000 -0.145 0.884     0.177
flexural_std             0.000 -0.682 0.495     1.014
flexural_max             0.000 -0.275 0.783     0.352
flexural_min             0.000  0.327 0.743     0.428
flexural_peak_to_peak    0.000 -0.301 0.763     0.389
flexural_rms             0.000 -0.682 0.495     1.014
flexural_kurtosis        0.000  0.760 0.447     1.161
torsional_energy         0.000 -0.275 0.783     0.352
flexural_energy          0.000 -0.477 0.633     0.659
energy_ratio             0.000  0.549 0.583     0.779
---
Concordance = 0.772
Partial AIC = 104.719
log-likelihood ratio test = 7.961 on 17 df
-log2(p) of ll-ratio test = 0.048


Survival Analysis model building completed.

--- Saving Models and Scalers ---
Scaler 'Frequency_1_features' saved.
Scaler 'Frequency_1_windows' saved.
Scaler 'Frequency_2_features' saved.
Scaler 'Frequency_2_windows' saved.
Scaler 'Frequency_3_features' saved.
Scaler 'Frequency_3_windows' saved.
Scaler 'Frequency_4_features' saved.
Scaler 'Frequency_4_windows' saved.
Scaler 'Frequency_5_features' saved.
Scaler 'Frequency_5_windows' saved.
Model 'Frequency_1_isolation_forest' saved as joblib.
Model 'Frequency_1_ocsvm' saved as joblib.
Model 'Frequency_1_autoencoder' saved as Keras format.
Model 'Frequency_1_lstm_autoencoder' saved as Keras format.
Model 'Frequency_2_isolation_forest' saved as joblib.
Model 'Frequency_2_ocsvm' saved as joblib.
Model 'Frequency_2_autoencoder' saved as Keras format.
Model 'Frequency_2_lstm_autoencoder' saved as Keras format.
Model 'Frequency_3_isolation_forest' saved as joblib.
Model 'Frequency_3_ocsvm' saved as joblib.
Model 'Frequency_3_autoencoder' sa